In [1]:
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies lightautoml==0.3.8
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies pandas==2.0.3

Looking in links: /kaggle/input/lightautoml-038-dependecies
Processing /kaggle/input/lightautoml-038-dependecies/lightautoml-0.3.8-py3-none-any.whl
Processing /kaggle/input/lightautoml-038-dependecies/AutoWoE-1.3.2-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/cmaes-0.10.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/joblib-1.2.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/json2html-1.3.0.tar.gz (from lightautoml==0.3.8)
  Preparing metadata (setup.py) ... - \ done
Processing /kaggle/input/lightautoml-038-dependecies/lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/poetry_core-1.8.1-py3-none-any.whl (from lig

# My LGBM

In [2]:
%matplotlib inline
import gc
import os
import itertools
import pickle
import re
import time
from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from functools import reduce
from itertools import cycle
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn import metrics, model_selection, preprocessing, linear_model, ensemble, decomposition, tree
import lightgbm as lgb
import copy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import random
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from itertools import combinations  
from scipy.stats import skew
import copy
import joblib
from sklearn.decomposition import PCA
import warnings  
import pickle
from collections import defaultdict
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
import torch
warnings.filterwarnings('ignore', category=UserWarning)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:
INPUT_DIR = '../input/linking-writing-processes-to-writing-quality'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
train_essays = pd.read_csv('../input/writing-quality-challenge-constructed-essays/train_essays_02.csv')
train_essays.index = train_essays["Unnamed: 0"]
train_essays.index.name = None
train_essays.drop(columns=["Unnamed: 0"], inplace=True)
train_essays_with_upper = pd.read_csv('/kaggle/input/essays-generator-with-upper/essays_with_upper.csv') 
train_essays_with_upper.index = train_essays_with_upper["Unnamed: 0"]
train_essays_with_upper.index.name = None
train_essays_with_upper.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
def preprocessing(df,dataset='test'):
    if dataset == 'train':
        add_value = 66231-17831+500
        df.loc[(df['id']=='a0c24719')&(df['event_id']>68),'down_time'] += add_value
        df.loc[(df['id']=='a0c24719')&(df['event_id']>68),'up_time'] += add_value
    for i in range(1,4):
        df[f'down_event_shift{i}'] = df.groupby('id')['down_event'].shift(i)
    df['need_drop'] = np.zeros(len(df))
    df.loc[(df['down_event']=='Shift')&(df['down_event_shift1']=='Shift')&(df['down_event_shift2']=='Shift')&(df['down_event_shift3']=='Shift'),'need_drop'] = 1
    df = df[df['need_drop']==0]
    df['event_id'] = df.groupby('id').cumcount()
    return df.drop(columns=['down_event_shift1','down_event_shift2','down_event_shift3','need_drop'])
train_logs = preprocessing(train_logs,dataset='train')
test_logs = preprocessing(test_logs)

/tmp/ipykernel_25/4222491666.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['event_id'] = df.groupby('id').cumcount()


In [5]:
#essay生成：普通版本和带大写版本
class EssayConstructor:
    
    def processingInputs(self,currTextInput):
        # Where the essay content will be stored
        essayText = ""
        # Produces the essay
        for Input in currTextInput.values:
            # Input[0] = activity
            # Input[1] = cursor_position
            # Input[2] = text_change
            # Input[3] = id
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]              
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')              
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]              
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue                
                
            # If activity = input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        return essayText
            
            
    def getEssays(self,df):
        # Copy required columns
        textInputDf = copy.deepcopy(df[['id', 'activity', 'cursor_position', 'text_change']])
        # Get rid of text inputs that make no change
        textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']     
        # construct essay, fast 
        tqdm.pandas()
        essay=textInputDf.groupby('id')[['activity','cursor_position', 'text_change']].progress_apply(lambda x: self.processingInputs(x))      
        # to dataframe
        essayFrame=essay.to_frame().reset_index()
        essayFrame.columns=['id','essay']
        # Returns the essay series
        return essayFrame

def getEssays_with_upper(df):
    df['down_event_shift'] = df.groupby('id')['down_event'].shift(1)
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change','down_event','down_event_shift']]
    valCountsArr = textInputDf['id'].value_counts(sort=False).values
    lastIndex = 0
    essaySeries = pd.Series()
    for index, valCount in enumerate(tqdm(valCountsArr)):
        capital = False
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change','down_event','down_event_shift']].iloc[lastIndex : lastIndex + valCount]
        lastIndex += valCount
        essayText = ""
        for Input in currTextInput.values:
            if Input[3] == 'CapsLock':
                capital = not capital
            if Input[0] == 'Nonproduction':
                continue
            if Input[0] != 'Nonproduction':
                if (Input[0] == 'Replace')&(Input[4] == 'Shift'):
                    replaceTxt = Input[2].split(' => ')
                    essayText = essayText[:Input[1] - len(replaceTxt[1])] + (replaceTxt[1]).upper() +\
                    essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                    continue
                    
                if Input[0] == 'Replace':
                    replaceTxt = Input[2].split(' => ')
                    essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] +\
                    essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                    continue
                    
                if Input[0] == 'Paste':
                    essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                    continue
                if Input[0] == 'Remove/Cut':
                    essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                    continue
                if "M" in Input[0]:
                    croppedTxt = Input[0][10:]
                    splitTxt = croppedTxt.split(' To ')
                    valueArr = [item.split(', ') for item in splitTxt]
                    moveData = (int(valueArr[0][0][1:]), 
                                int(valueArr[0][1][:-1]), 
                                int(valueArr[1][0][1:]), 
                                int(valueArr[1][1][:-1]))
                    if moveData[0] != moveData[2]:
                        if moveData[0] < moveData[2]:
                            essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] +\
                            essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                        else:
                            essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] +\
                            essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                    continue
                if capital|((Input[4]=='Shift')&(Input[3]=='q')):
                    essayText = essayText[:Input[1] - len(Input[2])] + Input[2].upper() + essayText[Input[1] - len(Input[2]):]
                else:
                    essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        essaySeries[index] = essayText
    essaySeries.index =  textInputDf['id'].unique()
    return pd.DataFrame(essaySeries, columns=['essay'])

In [6]:
#两个分位数
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [7]:
def calculate_entropy(text):  
    # 统计每个字符的出现次数  
    char_count = {}  
    for char in text:  
        if char in char_count:  
            char_count[char] += 1  
        else:  
            char_count[char] = 1  
    probabilities = [float(char_count[char]) / len(text) for char in char_count]  
    entropy = -sum([p * math.log2(p) for p in probabilities])  
    return entropy  

In [8]:
class Preprocessor_v1:
    
    def __init__(self,seed,essays,essays_with_upper,train_scores=None,tokenizer=None,method='train',save_cols=None):
        self.seed = seed
        self.tokenizer = tokenizer
        self.train_scores = train_scores
        self.save_cols = save_cols
        self.essays = essays
        self.essays_with_upper = essays_with_upper
        self.method =method 
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/','@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 100]
        self.idf = defaultdict(float)
        self.text_changes_dict = {
            'q': 'q', 
            ' ': 'space', 
            'NoChange': 'NoChange', 
            '.': 'full_stop', 
            ',': 'comma', 
            '\n': 'newline', 
            "'": 'single_quote', 
            '"': 'double_quote', 
            '-': 'dash', 
            '?': 'question_mark', 
            ';': 'semicolon', 
            '=': 'equals', 
            '/': 'slash', 
            '\\': 'double_backslash', 
            ':': 'colon'
        }
        self.AGGREGATIONS =  ['nunique','count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']
        self.AGGREGATIONS2 = ['nunique', 'mean', 'std', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
        self.AGGREGATIONS3 = ['nunique', 'mean', 'std', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
        self.AGGREGATIONS4 = ['nunique', 'mean', 'std', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
        self.AGGREGATIONS5 = ['nunique', 'mean', 'std', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
                
            di["move_to"] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
                else:
                    di["move_to"] += v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret

    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret


    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf
            
        return ret

    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret
    
    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def calculate_pauses(self, df, pause_threshold=2000):
        # Compute IKI within each 'id' group
        df['IKI'] = df.groupby('id')['down_time'].diff()

        # Define pauses
        df['is_pause'] = (df['IKI'] > pause_threshold)

        # Compute statistics for IKI
        iki_stats = df.groupby('id')['IKI'].agg(['mean', 'median', 'std', 'max']).reset_index().rename(columns={
            'mean': 'iki_mean',
            'median': 'iki_median',
            'std': 'iki_std',
            'max': 'iki_max'
        })

        # Compute pause counts
        pause_counts = df.groupby('id')['is_pause'].sum().reset_index(name='pause_count')

        # Compute average pause time excluding NaNs
        pause_times = df[df['is_pause']].groupby('id')['IKI'].mean().reset_index(name='average_pause_time')

        # Compute total pause time for paragraph
        para_pause_duration = df.groupby('id').apply(lambda group: group['IKI'].where(group['text_change'] == '\n').sum()).reset_index(name='para_pause_duration')

        # Merge pause features
        pause_features = pause_counts.merge(pause_times, on='id', how='left')
        pause_features = pause_features.merge(para_pause_duration, on='id', how='left')
        pause_features = pause_features.merge(iki_stats, on='id', how='left')

        # Compute total IKI time and exclude NaNs
        total_time = df.groupby('id')['IKI'].sum().reset_index(name='total_time')
        
        # Merge the total time into pause_features
        pause_features = pause_features.merge(total_time, on='id', how='left')

        # Calculate pause time ratio
        pause_features['pause_time_ratio'] = pause_features['pause_count'] * pause_features['average_pause_time']
        pause_features['pause_time_ratio'] = pause_features['pause_time_ratio'] / pause_features['total_time'].replace(0, np.nan)

        # Calculate times between sentences within each 'id' group
        df['sentence_end_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'].isin(['.', '?', '!']))).reset_index(level=0, drop=True)

        # Calculate statistics for times between sentences
        between_sentences_stats = df.groupby('id')['sentence_end_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_between_sentences_IKI', 'std': 'std_between_sentences_IKI'})

        # Calculate within-word IKI for 'q' characters within each 'id'
        df['within_word_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'] == 'q')).reset_index(level=0, drop=True)

        # Calculate statistics for within-word IKI
        within_word_stats = df.groupby('id')['within_word_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_within_word_IKI', 'std': 'std_within_word_IKI'})

        # Calculate between-words IKI for spaces or punctuation followed by 'q'
        df['between_words_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'].shift().isin([' '] + self.punctuations) & (group['text_change'] == 'q'))).reset_index(level=0, drop=True)

        # Calculate statistics for between-words IKI
        between_words_stats = df.groupby('id')['between_words_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_between_words_IKI', 'std': 'std_between_words_IKI'})

        # Combine all the IKI related features into one DataFrame
        pause_features = pause_features.merge(between_sentences_stats, on='id', how='left')
        pause_features = pause_features.merge(within_word_stats, on='id', how='left')
        pause_features = pause_features.merge(between_words_stats, on='id', how='left')

        return pause_features

    def brute_force_agg(self,df):
        #bruteforce agg
        agg_fe_df = df.groupby("id")[['down_time', 'cursor_position', 'word_count']].agg(
            ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
        agg_fe_df.columns = ['_'.join(x) for x in agg_fe_df.columns]
        agg_fe_df = agg_fe_df.add_prefix("tmp_")
        agg_fe_df.reset_index(inplace=True)
        return agg_fe_df
    
    def duration_features(self,df):
        logs = copy.deepcopy(df)
        logs['up_time_lagged'] = logs.groupby('id')['up_time'].shift(1).fillna(logs['down_time'])
        logs['time_diff'] = abs(logs['down_time'] - logs['up_time_lagged']) / 1000

        group = logs.groupby('id')['time_diff']
        initial_pause = logs.groupby('id')['down_time'].first() / 1000
        pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
        pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
        pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
        pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
        pauses_3_sec = group.apply(lambda x: (x > 3).sum())
        data = pd.DataFrame({
            'id': logs['id'].unique(),
            'initial_pause': initial_pause,
            'pauses_half_sec': pauses_half_sec,
            'pauses_1_sec': pauses_1_sec,
            'pauses_1_half_sec': pauses_1_half_sec,
            'pauses_2_sec': pauses_2_sec,
            'pauses_3_sec': pauses_3_sec,
        }).reset_index(drop=True)
        return data
    
    def essay_CountVectorizer_and_tfidf(self):
        if self.method=='train':
            essaysdf = copy.deepcopy(self.essays['essay'])
            essaysdf = pd.DataFrame({'id': essaysdf.index, 'essay': essaysdf.values})
            merged_data = essaysdf.merge(self.train_scores, on='id')
            count_vectorizer = CountVectorizer(ngram_range=(1, 2))
            tokenizer = count_vectorizer.fit_transform(merged_data['essay'])
            y = merged_data['score']
            tokenizer = tokenizer.todense()
            count_vector = pd.DataFrame()
            for i in range(tokenizer.shape[1]) : 
                L = list(tokenizer[:,i])
                li = [int(x) for x in L ]
                count_vector[f'feature {i}'] = li
            df_index = essaysdf['id']
            count_vector.loc[:, 'id'] = df_index
            
            save_cols = []
            for i in count_vector.columns:
                if sum(count_vector[i]==0)/len(count_vector)<0.1:
                    save_cols.append(i)

            return count_vector[save_cols],count_vectorizer,save_cols

        else:
            essaysdf = copy.deepcopy(self.essays['essay'])
            essaysdf = pd.DataFrame({'id': essaysdf.index, 'essay': essaysdf.values})
            tokenizer = self.tokenizer.transform(essaysdf['essay'])
            tokenizer = tokenizer.todense()
            count_vector = pd.DataFrame()
            for i in range(tokenizer.shape[1]): 
                L = list(tokenizer[:,i])
                li = [int(x) for x in L ]
                count_vector[f'feature {i}'] = li
            df_index = essaysdf['id']
            count_vector.loc[:, 'id'] = df_index
            return count_vector[self.save_cols]       
        
    def other_features(self,df):
        a = pd.DataFrame()
        a['Input_all_ratio'] = df.groupby(['id']).apply(lambda x:sum(x['activity']!='Input'))/df.groupby(['id']).apply(lambda x:sum(x['activity']=='Input'))
        a['all_q_ratio'] = df.groupby(['id']).apply(lambda x:sum(x['down_event']!='q'))/df.groupby(['id']).apply(lambda x:sum(x['down_event']=='q'))
        activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        events_dict = {
                        'q':'q', 
                        'Space':'Space', 
                        'Backspace':'Backspace', 
                        'Shift':'Shift', 
                        'ArrowRight':'ArrowRight', 
                        'Leftclick':'Leftclick', 
                        'ArrowLeft':'ArrowLeft', 
                        '.':'fullstop', 
                        ',':'comma', 
                        'ArrowDown':'ArrowDown', 
                        'ArrowUp':'ArrowUp', 
                        'Enter':'Enter', 
                        'CapsLock':'CapsLock', 
                        "'":'single_quote', 
                        'Delete':'Delete', 
                        'Unidentified':'Unidentified',
                      }
        for i in tqdm(activities):
            for j in events_dict:
                a[f'{i}_{events_dict[j]}_count'] = df.groupby('id').apply(lambda x:len(x[(x['activity']==i)&(x['down_event']==j)]))
        return a.reset_index()

    def language_error(self,df):
        a = pd.DataFrame()
        df['down_event_shift'] = df.groupby('id')['down_event'].shift(-1)
        letter_upper = df.groupby('id').apply(lambda x:len(x[(x['down_event']=='CapsLock')|((x['down_event']=='Shift')&(x['down_event_shift']=='q'))]))
        a['letter_big_count'] = letter_upper.values
        a['id'] = df['id'].unique()

        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index

        #避免将qqq.).切分成多个句子
        #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))

        essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
        essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())    
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
        essay_df = essay_df[essay_df['sent_len']!=0]
        errors_num = (essay_df.groupby('id').apply(len)-letter_upper).values
        a['error_num'] = errors_num                          #如果句子个数大于大写字母按键次数，那么文章会有语法错误

        return a 

    def sentence_error(self):
        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index
        essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
        essay_df = essay_df.explode('paragraph')
        # Number of characters in paragraphs
        essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
        essay_df = essay_df[essay_df['paragraph_len']!=0]
        essay_df['only_space'] = essay_df['paragraph'].apply(lambda x:'q' not in x)
        essay_df = essay_df[essay_df['only_space']==False]
        a = pd.DataFrame()
        a['para_error'] = essay_df.groupby('id').apply(lambda x:len(x[x['paragraph_len']<25]))   #一个段落字符过少可能不是完整的一句话，可能存在语法错误

        return a.reset_index()

    def language_error_letter(self):
        essay_df = copy.deepcopy(self.essays_with_upper)
        essay_df['id'] = essay_df.index

        #避免将qqq.).切分成多个句子
        #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))

        essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
        essay_df['sent'] = essay_df['sent'].apply(lambda x: str(x).replace('\n','').strip()) 
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
        essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
        essay_df['language_error_letter'] = essay_df['sent'].apply(lambda x:x[0])
        essay_df['if_q'] = essay_df['language_error_letter'].apply(lambda x:x.lower()=='q')
        essay_df = essay_df[essay_df['if_q']==True]
        a = pd.DataFrame()
        a['language_error_letter'] = essay_df.groupby('id').apply(lambda x:len(x[x['language_error_letter']=='q']))
        return a.reset_index()

    def R_burst(self,df):
        a = pd.DataFrame()
        df = df[(df['activity']=='Input')|(df['activity']=='Remove/Cut')].reset_index(drop=True)
        df['activity_shift'] = df.groupby('id')['activity'].shift().fillna(method='bfill')
        df['is_R_burst'] = df['activity'] != df['activity_shift']
        a['revision_count'] = df.groupby('id').apply(lambda x:x['is_R_burst'].sum())
        df['keystroke_duration'] = df.groupby('id')['down_time'].diff()
        df = df[df['keystroke_duration'].notnull()]

        a['revision_count_above2s'] = df.groupby('id').apply(lambda x:x[(x['is_R_burst']==True)&(x['keystroke_duration']>2)]['is_R_burst'].sum()).values
        Rburst =  df[(df['is_R_burst']==True)&(df['keystroke_duration']>2)]   #&(df['keystroke_duration']>2)
        Rburst_statistic = Rburst.groupby('id').agg({'keystroke_duration':['mean','max','sum','median']})
        Rburst_statistic.columns = ['_'.join(x) for x in Rburst_statistic.columns]
        
        return a.merge(Rburst_statistic.reset_index(),on='id',how='left')


    def split_essays_into_sentences(self):
        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index

        #避免将qqq.).切分成多个句子
        #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))
        #避免将类似于i.e.切分成多个句子
        essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
        essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())    #strip会删除字符串两端的空格
        # Number of characters in sentences
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))

        # Number of words in sentences
        essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
        essay_df['sent_word_count_diff'] = essay_df.groupby(['id'])['sent_word_count'].transform(lambda x:np.abs(x.diff()))
        essay_df['words_len_above10'] = essay_df['sent'].apply(lambda x: x.split(' '))
        essay_df['words_len_above10'] = essay_df['words_len_above10'].apply(lambda x:sum(len(y)>10 for y in x))

        essay_df['words_len_5-10'] = essay_df['sent'].apply(lambda x: x.split(' '))
        essay_df['words_len_5-10'] = essay_df['words_len_5-10'].apply(lambda x:sum(5<=len(y)<=10 for y in x))

        essay_df['words_len_first'] = essay_df['sent'].apply(lambda x: x.split(' '))
        essay_df['words_len_first'] = essay_df['words_len_first'].apply(lambda x:len(x[0]))

        essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
        return essay_df
    
    def compute_sentence_aggregations(self,df):
        sent_agg_df = pd.concat(
            [df[['id','sent_len']].groupby(['id']).agg(self.AGGREGATIONS),
             df[['id','sent_word_count']].groupby(['id']).agg(self.AGGREGATIONS),
             df[['id','sent_word_count_diff']].groupby(['id']).agg(self.AGGREGATIONS2),
             df[['id','words_len_above10']].groupby(['id']).agg(self.AGGREGATIONS3),
             df[['id','words_len_first']].groupby(['id']).agg(self.AGGREGATIONS4),
             df[['id','words_len_5-10']].groupby(['id']).agg(self.AGGREGATIONS5),

             ],
             axis=1)
        sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
        sent_agg_df['id'] = sent_agg_df.index    
        sent_agg_df = sent_agg_df.reset_index(drop=True)
        sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
        sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
        return sent_agg_df

    def split_essays_into_paragraphs(self):
        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index
        essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
        essay_df = essay_df.explode('paragraph')
        # Number of characters in paragraphs
        essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
        
        # Number of words in paragraphs
        essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
        essay_df['paragraph_word_count_diff'] = essay_df.groupby(['id'])['paragraph_word_count'].transform(lambda x:np.abs(x.diff()))

        essay_df['para_words_len_above10'] = essay_df['paragraph'].apply(lambda x: x.split(' '))
        essay_df['para_words_len_above10'] = essay_df['para_words_len_above10'].apply(lambda x:sum(len(y)>10 for y in x))

        essay_df['para_words_len_5-10'] = essay_df['paragraph'].apply(lambda x: x.split(' '))
        essay_df['para_words_len_5-10'] = essay_df['para_words_len_5-10'].apply(lambda x:sum(5<=len(y)<=10 for y in x))

        essay_df['para_words_len_first'] = essay_df['paragraph'].apply(lambda x: x.split(' '))
        essay_df['para_words_len_first'] = essay_df['para_words_len_first'].apply(lambda x:len(x[0]))
        
        essay_df['num_question'] = essay_df['paragraph'].apply(lambda x: len(re.findall(r'\?', x)))
        essay_df['num_yinyong'] = essay_df['paragraph'].apply(lambda x: len(re.findall(r'\"', x)))

        essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
        #有些段落可能全部是空格，类似于：'    '
        #essay_df['only_space'] = essay_df['paragraph'].apply(lambda x:'q' not in x)
        #essay_df = essay_df[essay_df['only_space']==False]
        return essay_df

    def compute_paragraph_aggregations(self,df):
        paragraph_agg_df = pd.concat(
            [df[['id','paragraph_len']].groupby(['id']).agg(self.AGGREGATIONS),\
             df[['id','paragraph_word_count']].groupby(['id']).agg(self.AGGREGATIONS),
             df[['id','paragraph_word_count_diff']].groupby(['id']).agg(self.AGGREGATIONS2),
             df[['id','para_words_len_above10']].groupby(['id']).agg(self.AGGREGATIONS3),
             df[['id','para_words_len_first']].groupby(['id']).agg(self.AGGREGATIONS4),
             df[['id','para_words_len_5-10']].groupby(['id']).agg(self.AGGREGATIONS5),
             df[['id','num_question']].groupby(['id']).agg(self.AGGREGATIONS5),
             df[['id','num_yinyong']].groupby(['id']).agg(self.AGGREGATIONS5),

             ], axis=1) 
        paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
        paragraph_agg_df['id'] = paragraph_agg_df.index
        paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
        paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
        paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
        return paragraph_agg_df
    
    def difficulty(self):
        df = copy.deepcopy(self.essays)
        df['token'] = [word_tokenize(p) for p in df["essay"]]
        df['token_len'] = df['token'].apply(lambda x : list(len(word) for word in x))
        df['verylong']  = df['token_len'].apply(lambda x : sum(c>=9 for c in x))
        df['long']      = df['token_len'].apply(lambda x : sum(c==7 or c==8 for c in x))
        df['mid']       = df['token_len'].apply(lambda x : sum(c==5 or c==6 for c in x))
        df['difficulty'] = df['verylong']*5 + df['long']*3 + df['mid']*1
        df['long_words'] = df['verylong']+df['long']
        df.reset_index(inplace=True)
        df.rename(columns={'index':'id'},inplace=True)

        #sentence
        df_sentence = copy.deepcopy(self.essays)
        df_sentence['id'] = df_sentence.index
        #避免将类似于i.e.切分成多个句子
        df_sentence['essay'] = df_sentence['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        df_sentence['sent'] = df_sentence['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        df_sentence = df_sentence.explode('sent')   #explode将列表里的元素展开
        df_sentence['sent'] = df_sentence['sent'].apply(lambda x: x.replace('\n','').strip())    #strip会删除字符串两端的空格
        # Number of characters in sentences
        df_sentence['sent_len'] = df_sentence['sent'].apply(lambda x: len(x))
        df_sentence['sent_word_count'] = df_sentence['sent'].apply(lambda x: len(x.split(' ')))
        df_sentence = df_sentence[df_sentence['sent_len']!=0]

        df_sentence['sentence_token'] = [word_tokenize(p) for p in df_sentence["sent"]]
        df_sentence['sentence_token_len'] = df_sentence['sentence_token'].apply(lambda x : list(len(word) for word in x))
        df_sentence['sentence_verylong']  = df_sentence['sentence_token_len'].apply(lambda x : sum(c>=9 for c in x))
        df_sentence['sentence_long']      = df_sentence['sentence_token_len'].apply(lambda x : sum(c==7 or c==8 for c in x))
        df_sentence['sentence_mid']       = df_sentence['sentence_token_len'].apply(lambda x : sum(c==5 or c==6 for c in x))
        df_sentence['sentence_difficulty'] = df_sentence['sentence_verylong']*5 + df_sentence['sentence_long']*3 + df_sentence['sentence_mid']*1
        df_sentence['sentence_long_words'] = df_sentence['sentence_verylong']+df_sentence['sentence_long']
        a = df_sentence.groupby('id')[['sentence_verylong','sentence_long','sentence_mid','sentence_difficulty','sentence_long_words']].agg(['max','mean','sum'])
        a.columns = ['_'.join(x) for x in a.columns]

        return (df[['id','verylong','long','mid','difficulty','long_words']]).merge(a,on='id',how='left')
    
    def entropy(self):
        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index
        essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
        essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())    #strip会删除字符串两端的空格
        # Number of characters in sentences
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
        essay_df['complexity'] = essay_df['sent'].apply(lambda x:calculate_entropy(x))
        a = essay_df.groupby('id').agg({'complexity':['max','mean','std','sum','median']})
        a.columns = ['_'.join(i) for i in a.columns]
        return a.reset_index()
    
    def make_feats(self, df):
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        print("Engineering time data")
        for gap in self.gaps:
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
            
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)
        
        print("Engineering cursor position data")
        for gap in self.gaps:
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        print("Engineering word count data")
        for gap in self.gaps:
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)        
        
        print("Engineering statistical summaries for features")
        feats_stat = [
            ('event_id', ['max']),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ]
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print("Calculating pause features")
        tmp_df = self.calculate_pauses(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print('<merge brute force agg.>')
        tmp_df = self.brute_force_agg(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['tmp_word_count_max'] / feats['tmp_down_time_max']
        feats['word_event_ratio'] = feats['tmp_word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['tmp_down_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['tmp_down_time_max']

        print('<merge duration_features.>')
        tmp_df = self.duration_features(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        if self.method == 'train':
            feats = feats.merge(self.train_scores, on='id', how='left')
        
        print('<merge countvectorizer_and_tfidf_features.>')
        if self.method == 'train':
            tmp_df,tokenizer,save_cols = self.essay_CountVectorizer_and_tfidf()
        else:
            tmp_df = self.essay_CountVectorizer_and_tfidf()
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print('<merge other features.>')
        if self.method == 'train':
            if os.path.exists('/kaggle/input/lgbm-and-nn-on-sentences'):  
                tmp_df = pd.read_csv('/kaggle/input/lgbm-and-nn-on-sentences/train_agg_ratio.csv')
            else:
                tmp_df = self.other_features(df)
        else:
            tmp_df = self.other_features(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print('<merge errors features.>')
        tmp_df = self.language_error(df)
        feats = feats.merge(tmp_df, on='id', how='left')
        tmp_df = self.sentence_error()
        feats = feats.merge(tmp_df, on='id', how='left')
        if self.method == 'train':
            tmp_df =  self.language_error_letter()
        else:
            essays_upper = getEssays_with_upper(df)
            tmp_df =  self.language_error_letter()
        feats = feats.merge(tmp_df, on='id', how='left')
        
        print('merge sentence and paragraph agg features')
        sent_df = self.split_essays_into_sentences()
        tmp_df = self.compute_sentence_aggregations(sent_df)
        feats = feats.merge(tmp_df, on='id', how='left')
        
        paragraph_df = self.split_essays_into_paragraphs()
        tmp_df = self.compute_paragraph_aggregations(paragraph_df)
        feats = feats.merge(tmp_df, on='id', how='left')

        print('merge R burst features')
        tmp_df = self.R_burst(df)
        feats = feats.merge(tmp_df, on='id', how='left')

        print('merge difficulty agg features')
        tmp_df = self.difficulty()
        feats = feats.merge(tmp_df, on='id', how='left')
        
        #print('merge sentence entropy features')
        #tmp_df = self.entropy()
        #feats = feats.merge(tmp_df, on='id', how='left')
        
        if self.method == 'train':
            return feats,tokenizer,save_cols
        else:
            return feats
        
        

class Preprocessor_v2:
    
    def __init__(self,seed,essays,essays_with_upper,train_scores=None,tokenizer=None,method='train',save_cols=None):
        self.seed = seed
        self.tokenizer = tokenizer
        self.train_scores = train_scores
        self.save_cols = save_cols
        self.essays = essays
        self.essays_with_upper = essays_with_upper
        self.method =method 
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 100]
        self.idf = defaultdict(float)
        self.text_changes_dict = {
            'q': 'q', 
            ' ': 'space', 
            'NoChange': 'NoChange', 
            '.': 'full_stop', 
            ',': 'comma', 
            '\n': 'newline', 
            "'": 'single_quote', 
            '"': 'double_quote', 
            '-': 'dash', 
            '?': 'question_mark', 
            ';': 'semicolon', 
            '=': 'equals', 
            '/': 'slash', 
            '\\': 'double_backslash', 
            ':': 'colon'
        }
        self.AGGREGATIONS =  ['nunique','count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']
        self.AGGREGATIONS2 = ['nunique', 'mean', 'std', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
        self.AGGREGATIONS3 = ['nunique', 'mean', 'std', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
        self.AGGREGATIONS4 = ['nunique', 'mean', 'std', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
        self.AGGREGATIONS5 = ['nunique', 'mean', 'std', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
                
            di["move_to"] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
                else:
                    di["move_to"] += v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret

    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret


    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf
            
        return ret

    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret
    
    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def calculate_pauses(self, df, pause_threshold=2000):
        # Compute IKI within each 'id' group
        df['IKI'] = df.groupby('id')['down_time'].diff()

        # Define pauses
        df['is_pause'] = (df['IKI'] > pause_threshold)

        # Compute statistics for IKI
        iki_stats = df.groupby('id')['IKI'].agg(['mean', 'median', 'std', 'max']).reset_index().rename(columns={
            'mean': 'iki_mean',
            'median': 'iki_median',
            'std': 'iki_std',
            'max': 'iki_max'
        })

        # Compute pause counts
        pause_counts = df.groupby('id')['is_pause'].sum().reset_index(name='pause_count')

        # Compute average pause time excluding NaNs
        pause_times = df[df['is_pause']].groupby('id')['IKI'].mean().reset_index(name='average_pause_time')

        # Compute total pause time for paragraph
        para_pause_duration = df.groupby('id').apply(lambda group: group['IKI'].where(group['text_change'] == '\n').sum()).reset_index(name='para_pause_duration')

        # Merge pause features
        pause_features = pause_counts.merge(pause_times, on='id', how='left')
        pause_features = pause_features.merge(para_pause_duration, on='id', how='left')
        pause_features = pause_features.merge(iki_stats, on='id', how='left')

        # Compute total IKI time and exclude NaNs
        total_time = df.groupby('id')['IKI'].sum().reset_index(name='total_time')
        
        # Merge the total time into pause_features
        pause_features = pause_features.merge(total_time, on='id', how='left')

        # Calculate pause time ratio
        pause_features['pause_time_ratio'] = pause_features['pause_count'] * pause_features['average_pause_time']
        pause_features['pause_time_ratio'] = pause_features['pause_time_ratio'] / pause_features['total_time'].replace(0, np.nan)

        # Calculate times between sentences within each 'id' group
        df['sentence_end_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'].isin(['.', '?', '!']))).reset_index(level=0, drop=True)

        # Calculate statistics for times between sentences
        between_sentences_stats = df.groupby('id')['sentence_end_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_between_sentences_IKI', 'std': 'std_between_sentences_IKI'})

        # Calculate within-word IKI for 'q' characters within each 'id'
        df['within_word_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'] == 'q')).reset_index(level=0, drop=True)

        # Calculate statistics for within-word IKI
        within_word_stats = df.groupby('id')['within_word_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_within_word_IKI', 'std': 'std_within_word_IKI'})

        # Calculate between-words IKI for spaces or punctuation followed by 'q'
        df['between_words_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'].shift().isin([' '] + self.punctuations) & (group['text_change'] == 'q'))).reset_index(level=0, drop=True)

        # Calculate statistics for between-words IKI
        between_words_stats = df.groupby('id')['between_words_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_between_words_IKI', 'std': 'std_between_words_IKI'})

        # Combine all the IKI related features into one DataFrame
        pause_features = pause_features.merge(between_sentences_stats, on='id', how='left')
        pause_features = pause_features.merge(within_word_stats, on='id', how='left')
        pause_features = pause_features.merge(between_words_stats, on='id', how='left')

        return pause_features

    def brute_force_agg(self,df):
        #bruteforce agg
        agg_fe_df = df.groupby("id")[['down_time', 'cursor_position', 'word_count']].agg(
            ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
        agg_fe_df.columns = ['_'.join(x) for x in agg_fe_df.columns]
        agg_fe_df = agg_fe_df.add_prefix("tmp_")
        agg_fe_df.reset_index(inplace=True)
        return agg_fe_df
    
    def duration_features(self,df):
        logs = copy.deepcopy(df)
        logs['up_time_lagged'] = logs.groupby('id')['up_time'].shift(1).fillna(logs['down_time'])
        logs['time_diff'] = abs(logs['down_time'] - logs['up_time_lagged']) / 1000

        group = logs.groupby('id')['time_diff']
        initial_pause = logs.groupby('id')['down_time'].first() / 1000
        pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
        pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
        pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
        pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
        pauses_3_sec = group.apply(lambda x: (x > 3).sum())
        data = pd.DataFrame({
            'id': logs['id'].unique(),
            'initial_pause': initial_pause,
            'pauses_half_sec': pauses_half_sec,
            'pauses_1_sec': pauses_1_sec,
            'pauses_1_half_sec': pauses_1_half_sec,
            'pauses_2_sec': pauses_2_sec,
            'pauses_3_sec': pauses_3_sec,
        }).reset_index(drop=True)
        return data
    
    def essay_CountVectorizer_and_tfidf(self):
        if self.method=='train':
            essaysdf = copy.deepcopy(self.essays['essay'])
            essaysdf = pd.DataFrame({'id': essaysdf.index, 'essay': essaysdf.values})
            merged_data = essaysdf.merge(self.train_scores, on='id')
            count_vectorizer = CountVectorizer(ngram_range=(1, 2))
            tokenizer = count_vectorizer.fit_transform(merged_data['essay'])
            y = merged_data['score']
            tokenizer = tokenizer.todense()
            count_vector = pd.DataFrame()
            for i in range(tokenizer.shape[1]) : 
                L = list(tokenizer[:,i])
                li = [int(x) for x in L ]
                count_vector[f'feature {i}'] = li
            df_index = essaysdf['id']
            count_vector.loc[:, 'id'] = df_index
            
            save_cols = []
            for i in count_vector.columns:
                if sum(count_vector[i]==0)/len(count_vector)<0.1:
                    save_cols.append(i)
            return count_vector[save_cols],count_vectorizer,save_cols

        else:
            essaysdf = copy.deepcopy(self.essays['essay'])
            essaysdf = pd.DataFrame({'id': essaysdf.index, 'essay': essaysdf.values})
            tokenizer = self.tokenizer.transform(essaysdf['essay'])
            tokenizer = tokenizer.todense()
            count_vector = pd.DataFrame()
            for i in range(tokenizer.shape[1]): 
                L = list(tokenizer[:,i])
                li = [int(x) for x in L ]
                count_vector[f'feature {i}'] = li
            df_index = essaysdf['id']
            count_vector.loc[:, 'id'] = df_index
            return count_vector[self.save_cols]       
        
    def other_features(self,df):
        a = pd.DataFrame()
        a['Input_all_ratio'] = df.groupby(['id']).apply(lambda x:sum(x['activity']!='Input'))/df.groupby(['id']).apply(lambda x:sum(x['activity']=='Input'))
        a['all_q_ratio'] = df.groupby(['id']).apply(lambda x:sum(x['down_event']!='q'))/df.groupby(['id']).apply(lambda x:sum(x['down_event']=='q'))
        activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        events_dict = {
                        'q':'q', 
                        'Space':'Space', 
                        'Backspace':'Backspace', 
                        'Shift':'Shift', 
                        'ArrowRight':'ArrowRight', 
                        'Leftclick':'Leftclick', 
                        'ArrowLeft':'ArrowLeft', 
                        '.':'fullstop', 
                        ',':'comma', 
                        'ArrowDown':'ArrowDown', 
                        'ArrowUp':'ArrowUp', 
                        'Enter':'Enter', 
                        'CapsLock':'CapsLock', 
                        "'":'single_quote', 
                        'Delete':'Delete', 
                        'Unidentified':'Unidentified',
                      }
        for i in tqdm(activities):
            for j in events_dict:
                a[f'{i}_{events_dict[j]}_count'] = df.groupby('id').apply(lambda x:len(x[(x['activity']==i)&(x['down_event']==j)]))
        return a.reset_index()

    def language_error(self,df):
        a = pd.DataFrame()
        df['down_event_shift'] = df.groupby('id')['down_event'].shift(-1)
        letter_upper = df.groupby('id').apply(lambda x:len(x[(x['down_event']=='CapsLock')|((x['down_event']=='Shift')&(x['down_event_shift']=='q'))]))
        a['letter_big_count'] = letter_upper.values
        a['id'] = df['id'].unique()

        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index

        #避免将qqq.).切分成多个句子
        #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))

        essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
        essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())    
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
        essay_df = essay_df[essay_df['sent_len']!=0]
        errors_num = (essay_df.groupby('id').apply(len)-letter_upper).values
        a['error_num'] = errors_num                          #如果句子个数大于大写字母按键次数，那么文章会有语法错误

        return a 

    def sentence_error(self):
        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index
        essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
        essay_df = essay_df.explode('paragraph')
        # Number of characters in paragraphs
        essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
        essay_df = essay_df[essay_df['paragraph_len']!=0]
        essay_df['only_space'] = essay_df['paragraph'].apply(lambda x:'q' not in x)
        essay_df = essay_df[essay_df['only_space']==False]
        a = pd.DataFrame()
        a['para_error'] = essay_df.groupby('id').apply(lambda x:len(x[x['paragraph_len']<25]))   #一个段落字符过少可能不是完整的一句话，可能存在语法错误

        return a.reset_index()

    def language_error_letter(self):
        essay_df = copy.deepcopy(self.essays_with_upper)
        essay_df['id'] = essay_df.index

        #避免将qqq.).切分成多个句子
        #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))

        essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
        essay_df['sent'] = essay_df['sent'].apply(lambda x: str(x).replace('\n','').strip()) 
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
        essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
        essay_df['language_error_letter'] = essay_df['sent'].apply(lambda x:x[0])
        essay_df['if_q'] = essay_df['language_error_letter'].apply(lambda x:x.lower()=='q')
        essay_df = essay_df[essay_df['if_q']==True]
        a = pd.DataFrame()
        a['language_error_letter'] = essay_df.groupby('id').apply(lambda x:len(x[x['language_error_letter']=='q']))
        return a.reset_index()

    def R_burst(self,df):
        a = pd.DataFrame()
        df = df[(df['activity']=='Input')|(df['activity']=='Remove/Cut')].reset_index(drop=True)
        df['activity_shift'] = df.groupby('id')['activity'].shift().fillna(method='bfill')
        df['is_R_burst'] = df['activity'] != df['activity_shift']
        a['revision_count'] = df.groupby('id').apply(lambda x:x['is_R_burst'].sum())
        df['keystroke_duration'] = df.groupby('id')['down_time'].diff()
        df = df[df['keystroke_duration'].notnull()]

        a['revision_count_above2s'] = df.groupby('id').apply(lambda x:x[(x['is_R_burst']==True)&(x['keystroke_duration']>2)]['is_R_burst'].sum()).values
        Rburst =  df[(df['is_R_burst']==True)&(df['keystroke_duration']>2)]   #&(df['keystroke_duration']>2)
        Rburst_statistic = Rburst.groupby('id').agg({'keystroke_duration':['mean','max','sum','median']})
        Rburst_statistic.columns = ['_'.join(x) for x in Rburst_statistic.columns]
        
        return a.merge(Rburst_statistic.reset_index(),on='id',how='left')


    def split_essays_into_sentences(self):
        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index

        #避免将qqq.).切分成多个句子
        #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))
        #避免将类似于i.e.切分成多个句子
        essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
        essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())    #strip会删除字符串两端的空格
        # Number of characters in sentences
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))

        # Number of words in sentences
        essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
        essay_df['sent_word_count_diff'] = essay_df.groupby(['id'])['sent_word_count'].transform(lambda x:np.abs(x.diff()))
        essay_df['words_len_above10'] = essay_df['sent'].apply(lambda x: x.split(' '))
        essay_df['words_len_above10'] = essay_df['words_len_above10'].apply(lambda x:sum(len(y)>10 for y in x))

        essay_df['words_len_5-10'] = essay_df['sent'].apply(lambda x: x.split(' '))
        essay_df['words_len_5-10'] = essay_df['words_len_5-10'].apply(lambda x:sum(5<=len(y)<=10 for y in x))

        essay_df['words_len_first'] = essay_df['sent'].apply(lambda x: x.split(' '))
        essay_df['words_len_first'] = essay_df['words_len_first'].apply(lambda x:len(x[0]))

        essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
        return essay_df
    
    def compute_sentence_aggregations(self,df):
        sent_agg_df = pd.concat(
            [df[['id','sent_len']].groupby(['id']).agg(self.AGGREGATIONS),
             df[['id','sent_word_count']].groupby(['id']).agg(self.AGGREGATIONS),
             df[['id','sent_word_count_diff']].groupby(['id']).agg(self.AGGREGATIONS2),
             df[['id','words_len_above10']].groupby(['id']).agg(self.AGGREGATIONS3),
             df[['id','words_len_first']].groupby(['id']).agg(self.AGGREGATIONS4),
             df[['id','words_len_5-10']].groupby(['id']).agg(self.AGGREGATIONS5),

             ],
             axis=1)
        sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
        sent_agg_df['id'] = sent_agg_df.index    
        sent_agg_df = sent_agg_df.reset_index(drop=True)
        sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
        sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
        return sent_agg_df

    def split_essays_into_paragraphs(self):
        essay_df = copy.deepcopy(self.essays)
        essay_df['id'] = essay_df.index
        essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
        essay_df = essay_df.explode('paragraph')
        # Number of characters in paragraphs
        essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
        
        # Number of words in paragraphs
        essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
        essay_df['paragraph_word_count_diff'] = essay_df.groupby(['id'])['paragraph_word_count'].transform(lambda x:np.abs(x.diff()))

        essay_df['para_words_len_above10'] = essay_df['paragraph'].apply(lambda x: x.split(' '))
        essay_df['para_words_len_above10'] = essay_df['para_words_len_above10'].apply(lambda x:sum(len(y)>10 for y in x))

        essay_df['para_words_len_5-10'] = essay_df['paragraph'].apply(lambda x: x.split(' '))
        essay_df['para_words_len_5-10'] = essay_df['para_words_len_5-10'].apply(lambda x:sum(5<=len(y)<=10 for y in x))

        essay_df['para_words_len_first'] = essay_df['paragraph'].apply(lambda x: x.split(' '))
        essay_df['para_words_len_first'] = essay_df['para_words_len_first'].apply(lambda x:len(x[0]))
        
        essay_df['num_question'] = essay_df['paragraph'].apply(lambda x: len(re.findall(r'\?', x)))
        essay_df['num_yinyong'] = essay_df['paragraph'].apply(lambda x: len(re.findall(r'\"', x)))

        essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
        #有些段落可能全部是空格，类似于：'    '
        #essay_df['only_space'] = essay_df['paragraph'].apply(lambda x:'q' not in x)
        #essay_df = essay_df[essay_df['only_space']==False]
        return essay_df

    def compute_paragraph_aggregations(self,df):
        paragraph_agg_df = pd.concat(
            [df[['id','paragraph_len']].groupby(['id']).agg(self.AGGREGATIONS),\
             df[['id','paragraph_word_count']].groupby(['id']).agg(self.AGGREGATIONS),
             df[['id','paragraph_word_count_diff']].groupby(['id']).agg(self.AGGREGATIONS2),
             df[['id','para_words_len_above10']].groupby(['id']).agg(self.AGGREGATIONS3),
             df[['id','para_words_len_first']].groupby(['id']).agg(self.AGGREGATIONS4),
             df[['id','para_words_len_5-10']].groupby(['id']).agg(self.AGGREGATIONS5),
             df[['id','num_question']].groupby(['id']).agg(self.AGGREGATIONS5),
             df[['id','num_yinyong']].groupby(['id']).agg(self.AGGREGATIONS5),

             ], axis=1) 
        paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
        paragraph_agg_df['id'] = paragraph_agg_df.index
        paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
        paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
        paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
        return paragraph_agg_df
    
    def difficulty(self):
        df = copy.deepcopy(self.essays)
        df['token'] = [word_tokenize(p) for p in df["essay"]]
        df['token_len'] = df['token'].apply(lambda x : list(len(word) for word in x))
        df['verylong']  = df['token_len'].apply(lambda x : sum(c>=9 for c in x))
        df['long']      = df['token_len'].apply(lambda x : sum(c==7 or c==8 for c in x))
        df['mid']       = df['token_len'].apply(lambda x : sum(c==5 or c==6 for c in x))
        df['difficulty'] = df['verylong']*5 + df['long']*3 + df['mid']*1
        df['long_words'] = df['verylong']+df['long']
        df.reset_index(inplace=True)
        df.rename(columns={'index':'id'},inplace=True)

        #sentence
        df_sentence = copy.deepcopy(self.essays)
        df_sentence['id'] = df_sentence.index
        #避免将类似于i.e.切分成多个句子
        df_sentence['essay'] = df_sentence['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
        df_sentence['sent'] = df_sentence['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        df_sentence = df_sentence.explode('sent')   #explode将列表里的元素展开
        df_sentence['sent'] = df_sentence['sent'].apply(lambda x: x.replace('\n','').strip())    #strip会删除字符串两端的空格
        # Number of characters in sentences
        df_sentence['sent_len'] = df_sentence['sent'].apply(lambda x: len(x))
        df_sentence['sent_word_count'] = df_sentence['sent'].apply(lambda x: len(x.split(' ')))
        df_sentence = df_sentence[df_sentence['sent_len']!=0]

        df_sentence['sentence_token'] = [word_tokenize(p) for p in df_sentence["sent"]]
        df_sentence['sentence_token_len'] = df_sentence['sentence_token'].apply(lambda x : list(len(word) for word in x))
        df_sentence['sentence_verylong']  = df_sentence['sentence_token_len'].apply(lambda x : sum(c>=9 for c in x))
        df_sentence['sentence_long']      = df_sentence['sentence_token_len'].apply(lambda x : sum(c==7 or c==8 for c in x))
        df_sentence['sentence_mid']       = df_sentence['sentence_token_len'].apply(lambda x : sum(c==5 or c==6 for c in x))
        df_sentence['sentence_difficulty'] = df_sentence['sentence_verylong']*5 + df_sentence['sentence_long']*3 + df_sentence['sentence_mid']*1
        df_sentence['sentence_long_words'] = df_sentence['sentence_verylong']+df_sentence['sentence_long']
        a = df_sentence.groupby('id')[['sentence_verylong','sentence_long','sentence_mid','sentence_difficulty','sentence_long_words']].agg(['max','mean','sum'])
        a.columns = ['_'.join(x) for x in a.columns]

        return (df[['id','verylong','long','mid','difficulty','long_words']]).merge(a,on='id',how='left')
    
    def make_feats(self, df):
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        print("Engineering time data")
        for gap in self.gaps:
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)
        
        print("Engineering cursor position data")
        for gap in self.gaps:
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        print("Engineering word count data")
        for gap in self.gaps:
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)        
        
        print("Engineering statistical summaries for features")
        feats_stat = [
            ('event_id', ['max']),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ]
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print("Calculating pause features")
        tmp_df = self.calculate_pauses(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print('<merge brute force agg.>')
        tmp_df = self.brute_force_agg(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['tmp_word_count_max'] / feats['tmp_down_time_max']
        feats['word_event_ratio'] = feats['tmp_word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['tmp_down_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['tmp_down_time_max']

        print('<merge duration_features.>')
        tmp_df = self.duration_features(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        if self.method == 'train':
            feats = feats.merge(self.train_scores, on='id', how='left')
        
        print('<merge countvectorizer_and_tfidf_features.>')
        if self.method == 'train':
            tmp_df,tokenizer,save_cols = self.essay_CountVectorizer_and_tfidf()
        else:
            tmp_df = self.essay_CountVectorizer_and_tfidf()
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print('<merge other features.>')
        if self.method == 'train':
            if os.path.exists('/kaggle/input/lgbm-and-nn-on-sentences'):  
                tmp_df = pd.read_csv('/kaggle/input/lgbm-and-nn-on-sentences/train_agg_ratio.csv')
            else:
                tmp_df = self.other_features(df)
        else:
            tmp_df = self.other_features(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print('<merge errors features.>')
        tmp_df = self.language_error(df)
        feats = feats.merge(tmp_df, on='id', how='left')
        tmp_df = self.sentence_error()
        feats = feats.merge(tmp_df, on='id', how='left')
        if self.method == 'train':
            tmp_df =  self.language_error_letter()
        else:
            essays_upper = getEssays_with_upper(df)
            tmp_df =  self.language_error_letter()
        feats = feats.merge(tmp_df, on='id', how='left')
        
        print('merge sentence and paragraph agg features')
        sent_df = self.split_essays_into_sentences()
        tmp_df = self.compute_sentence_aggregations(sent_df)
        feats = feats.merge(tmp_df, on='id', how='left')
        
        paragraph_df = self.split_essays_into_paragraphs()
        tmp_df = self.compute_paragraph_aggregations(paragraph_df)
        feats = feats.merge(tmp_df, on='id', how='left')

        print('merge R burst features')
        tmp_df = self.R_burst(df)
        feats = feats.merge(tmp_df, on='id', how='left')

        print('merge difficulty agg features')
        tmp_df = self.difficulty()
        feats = feats.merge(tmp_df, on='id', how='left')
        
        #print('merge sentence entropy features')
        #tmp_df = self.entropy()
        #feats = feats.merge(tmp_df, on='id', how='left')
        
        if self.method == 'train':
            return feats,tokenizer,save_cols
        else:
            return feats

In [9]:
#LGBM V1 train_feats and test_feats
preprocessor = Preprocessor_v1(42,train_essays,train_essays_with_upper,train_scores = train_scores)
train_feats,tokenizer,save_cols = preprocessor.make_feats(train_logs)

test_essays = EssayConstructor().getEssays(test_logs)
test_essays.set_index('id',inplace=True)
test_essays.index.name = None
test_essays_with_upper = getEssays_with_upper(test_logs)
preprocessor = Preprocessor_v1(42,test_essays,test_essays_with_upper,tokenizer=tokenizer,method='test',save_cols=save_cols)
test_feats = preprocessor.make_feats(test_logs)

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 29/29 [03:14<00:00,  6.72s/it, column=word_count_change100, method=kurt]


Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 5822.66it/s]


Engineering event counts data


...

<merge other features.>


100%|██████████| 5/5 [00:00<00:00,  7.89it/s]


<merge errors features.>


100%|██████████| 3/3 [00:00<00:00, 589.53it/s]


merge sentence and paragraph agg features
merge R burst features
merge difficulty agg features


In [10]:
target_col = ['score']
f_read = open('/kaggle/input/select-features/feats_dict.pkl', 'rb')
lgb_cols_v1 = pickle.load(f_read)
f_read.close()

In [11]:
def LGBM_train_and_test_v1(features,params):
    OOF_PREDS = np.zeros(len(train_feats))
    TEST_PREDS = np.zeros(len(test_feats))
    best_iters_dict = defaultdict(list)
    models_dict = {}
    scores = []
    test_predict_list = []
    best_params = params
    best_iterations = [340, 318, 325, 301, 361]
    for i in range(5): 
        seeds = [3,6,38,39,43]
        seed = seeds[i]
        kf = model_selection.KFold(n_splits=10, random_state=seed, shuffle=True)
        oof_valid_preds = np.zeros(train_feats.shape[0])
        X_test = test_feats
        for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):
            params = {
                "objective": "regression",
                "metric": "rmse",
                'random_state': 42,
                "n_estimators" : best_iterations[i],
                "verbosity": -1,
                **best_params
            }
            
            X_train_pre, y_train_pre = train_feats.iloc[train_idx][features], train_feats.iloc[train_idx][target_col]
            X_valid_pre, y_valid_pre = train_feats.iloc[valid_idx][features], train_feats.iloc[valid_idx][target_col]
            pre_model = lgb.LGBMRegressor(**params)
            pre_model.fit(X_train_pre, y_train_pre, eval_set=[(X_valid_pre, y_valid_pre)],verbose=100)
            imp_df = pd.DataFrame()
            imp_df["feature"] = features       
            imp_df["importance"] = pre_model.feature_importances_
            imp_df = imp_df.sort_values(by='importance',ascending=False)
            features_select = list(imp_df[imp_df['importance']!=0]['feature'].values)
            print('-'*50)
            
            X_train, y_train = train_feats.iloc[train_idx][features_select], train_feats.iloc[train_idx][target_col]
            X_valid, y_valid = train_feats.iloc[valid_idx][features_select], train_feats.iloc[valid_idx][target_col]
            model = lgb.LGBMRegressor(**params)    
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],verbose=100)
            best_iters_dict[str(seed)].append(model.best_iteration_)
            valid_predict = model.predict(X_valid)
            oof_valid_preds[valid_idx] = valid_predict
            OOF_PREDS[valid_idx] += valid_predict / len(seeds)
            test_predict = model.predict(X_test[features_select])
            TEST_PREDS += test_predict / len(seeds) / 10
            test_predict_list.append(test_predict)
            score = metrics.mean_squared_error(y_valid, valid_predict, squared=False)
            models_dict[f'{fold}_{i}'] = model
        oof_score = metrics.mean_squared_error(train_feats[target_col], oof_valid_preds, squared=False)
        scores.append(oof_score)
    return OOF_PREDS,TEST_PREDS

params1 =  { 'boosting_type': 'gbdt', 
            'metric': 'rmse',
            'reg_alpha': 0.003188447814669599, 
            'reg_lambda': 0.0010228604507564066, 
            'colsample_bytree': 0.5420247656839267, 
            'subsample': 0.9778252382803456, 
            'feature_fraction': 0.8,
            'bagging_freq': 1,
            'bagging_fraction': 0.75,
            'num_leaves': 19, 
            'learning_rate': 0.01716485155812008,
            'min_child_samples': 46}
OOF_PREDS_v1,TEST_PREDS_v1 = LGBM_train_and_test_v1(lgb_cols_v1,params1)   

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.5420247656839267 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=0.9778252382803456 will be ignored. Current value: bagging_fraction=0.75
[100]	valid_0's rmse: 0.570975
[200]	valid_0's rmse: 0.552935
[300]	valid_0's rmse: 0.552601
--------------------------------------------------
[100]	valid_0's rmse: 0.572846
[200]	valid_0's rmse: 0.557329
[300]	valid_0's rmse: 0.559589
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.5420247656839267 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=0.9778252382803456 will be ignored. Current value: bagging_fraction=0.75
[

In [12]:
print('OOF metric LGBM v1 = {:.5f}'.format(metrics.mean_squared_error(train_feats[target_col], OOF_PREDS_v1, squared=False)))

OOF metric LGBM v1 = 0.60246


In [13]:
preprocessor = Preprocessor_v2(42,train_essays,train_essays_with_upper,train_scores = train_scores)
train_feats,tokenizer,save_cols = preprocessor.make_feats(train_logs)

test_essays = EssayConstructor().getEssays(test_logs)
test_essays.set_index('id',inplace=True)
test_essays.index.name = None
test_essays_with_upper = getEssays_with_upper(test_logs)
preprocessor = Preprocessor_v2(42,test_essays,test_essays_with_upper,tokenizer=tokenizer,method='test',save_cols=save_cols)
test_feats = preprocessor.make_feats(test_logs)

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 29/29 [03:15<00:00,  6.75s/it, column=word_count_change100, method=kurt]


Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 5380.07it/s]


Engineering event counts data


...

<merge other features.>


100%|██████████| 5/5 [00:00<00:00,  7.80it/s]


<merge errors features.>


100%|██████████| 3/3 [00:00<00:00, 570.19it/s]


merge sentence and paragraph agg features
merge R burst features
merge difficulty agg features


In [14]:
null1 = train_feats.isnull().sum().sort_values(ascending=False) / len(train_feats)
drop1 = list(null1[null1>0.9].index)
for col in train_feats.columns:
    if train_feats[col].nunique()==1:
        drop1.append(col)
cols = [col for col in train_feats.columns if col!='id']
corr_under_001 = (abs(train_feats[cols].corr()['score']).sort_values()<0.01)
corr_sort = abs(train_feats[cols].corr()['score']).sort_values()
drop2 = list(corr_sort[corr_under_001].index)

drop_cols = ['id']
train_cols = [col for col in train_feats.columns if col not in target_col + drop_cols]    #for NN

drop_cols_lgb = []
for i in train_feats.columns:
    if 'action_time' in i:
        drop_cols_lgb.append(i)
    if 'up_time' in i:
        drop_cols_lgb.append(i)
#lgbm v2 features
lgb_cols_v2 = [col for col in train_feats.columns if col not in target_col + drop_cols+drop_cols_lgb+drop1+drop2] #for LGBM

In [15]:
def LGBM_train_and_test_v2(features,params):
    OOF_PREDS = np.zeros(len(train_feats))
    TEST_PREDS = np.zeros(len(test_feats))
    models_dict = {}
    scores = []
    test_predict_list = []
    best_params = params
    for i in range(5): 
        seeds = [3,6,38,39,43]
        seed = seeds[i]
        kf = model_selection.KFold(n_splits=10, random_state=seed, shuffle=True)
        oof_valid_preds = np.zeros(train_feats.shape[0])
        X_test = test_feats[train_cols]
        for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):

            params = {
                "objective": "regression",
                "metric": "rmse",
                **best_params
            }
            
            X_train, y_train = train_feats.iloc[train_idx][features], train_feats.iloc[train_idx][target_col]
            X_valid, y_valid = train_feats.iloc[valid_idx][features], train_feats.iloc[valid_idx][target_col]

            model = lgb.LGBMRegressor(**params)    
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],verbose=100)
            valid_predict = model.predict(X_valid)
            oof_valid_preds[valid_idx] = valid_predict
            OOF_PREDS[valid_idx] += valid_predict / len(seeds)
            test_predict = model.predict(X_test[features])
            TEST_PREDS += test_predict / len(seeds) / 10
            test_predict_list.append(test_predict)
            score = metrics.mean_squared_error(y_valid, valid_predict, squared=False)
            models_dict[f'{fold}_{i}'] = model
        oof_score = metrics.mean_squared_error(train_feats[target_col], oof_valid_preds, squared=False)
        scores.append(oof_score)
    return OOF_PREDS,TEST_PREDS
params2 = {'n_estimators': 1024,
         'learning_rate': 0.005,
         'metric': 'rmse',
         'random_state': 42,
         'force_col_wise': True,
         'verbosity': 0,}

OOF_PREDS_v2,TEST_PREDS_v2 = LGBM_train_and_test_v2(lgb_cols_v2,params2)   

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[100]	valid_0's rmse: 0.715597
[200]	valid_0's rmse: 0.622316
[300]	valid_0's rmse: 0.585275
[400]	valid_0's rmse: 0.575264
[500]	valid_0's rmse: 0.573256
[600]	valid_0's rmse: 0.573894
[700]	valid_0's rmse: 0.575023
[800]	valid_0's rmse: 0.577544
[900]	valid_0's rmse: 0.579043
[1000]	valid_0's rmse: 0.579474
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[100]	valid_0's rmse: 0.827335
[200]	valid_0's rmse: 0.725054
[300]	valid_0's rmse: 0.67702
[400]	valid_0's rmse: 0.651847
[500]	valid_0's rmse: 0.63833
[600]	valid_0's rmse: 0.630155
[700]	valid_0's rmse: 0.624507
[800]	valid_0's rmse: 0.621096
[900]	valid_0's rmse: 0.619846
[1000]	valid_0's rmse: 0.618866
[LightGBM] [Warning] Find whitespaces in feature_names, repl

In [16]:
print('OOF metric LGBM v2 = {:.5f}'.format(metrics.mean_squared_error(train_feats[target_col], OOF_PREDS_v2, squared=False)))

OOF metric LGBM v2 = 0.60429


In [17]:
best_sc = 1
for w in np.arange(0, 1.01, 0.001):
    sc = metrics.mean_squared_error(train_feats[target_col], 
                                    w * OOF_PREDS_v1 + (1-w) * OOF_PREDS_v2, 
                                    squared=False)
    if sc < best_sc:
        best_sc = sc
        best_w = w
print('Composition best W = {:.3f}'.format(best_w))

Composition best W = 0.583


In [18]:
W = [best_w,1 - best_w]
OOF_PREDS_lgbm_pandas = OOF_PREDS_v1 * W[0] + OOF_PREDS_v2 * W[1]
TEST_PREDS_lgbm_pandas = TEST_PREDS_v1 * W[0] + TEST_PREDS_v2 * W[1]
TEST_PREDS_lgbm_pandas

array([1.27922582, 1.25867622, 1.28349684])

In [19]:
print('OOF metric LGBM×2 = {:.5f}'.format(metrics.mean_squared_error(train_feats[target_col], OOF_PREDS_lgbm_pandas, squared=False)))

OOF metric LGBM×2 = 0.60053


# MLP

In [20]:
INPUT_DIR = '../input/linking-writing-processes-to-writing-quality'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')

In [21]:
# Helper functions
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)
def percentile(n):
    def percentile_(x):
        return x.quantile(n/100)
    percentile_.__name__ = 'pct_{:02.0f}'.format(n)
    return percentile_

In [22]:
def get_essay_aggregations(essay_df):
    cols_to_drop = ['essay']
    # Total essay length
    essay_df['essay_len'] = essay_df['essay'].apply(lambda x: len(x))
    essay_df = essay_df.drop(columns=cols_to_drop)
    return essay_df

def split_essays_into_words(essay_df):
    essay_df['word'] = essay_df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    essay_df = essay_df.explode('word')
    # Word length (number of characters in word)
    essay_df['word_len'] = essay_df['word'].apply(lambda x: len(x))
    essay_df = essay_df[essay_df['word_len'] != 0]
    return essay_df

def compute_word_aggregations(word_df):
    word_agg_df = word_df[['id','word_len']].groupby(['id']).agg(self.aggregations)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    # New features: computing the # of words whose length exceed word_l
    for word_l in [5, 6, 7, 8, 9, 10, 11, 12]:
        word_agg_df[f'word_len_ge_{word_l}_count'] = word_df[word_df['word_len'] >= word_l].groupby(['id']).count().iloc[:, 0]
        word_agg_df[f'word_len_ge_{word_l}_count'] = word_agg_df[f'word_len_ge_{word_l}_count'].fillna(0)
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df

def split_essays_into_sentences(essay_df):
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
    return essay_df

def compute_sentence_aggregations(sent_df):
    sent_agg_df = sent_df[['id','sent_len','sent_word_count']].groupby(['id']).agg(self.aggregations)
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    # New features: computing the # of sentences whose (character) length exceed sent_l
    for sent_l in [50, 60, 75, 100]:
        sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_df[sent_df['sent_len'] >= sent_l].groupby(['id']).count().iloc[:, 0]
        sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_agg_df[f'sent_len_ge_{sent_l}_count'].fillna(0)
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    return sent_agg_df

def split_essays_into_paragraphs(essay_df):
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    # Number of sentences in paragraphs
    essay_df['paragraph_sent_count'] = essay_df['paragraph'].apply(lambda x: len(x.split('\\.|\\?|\\!')))
    # Number of words in paragraphs
    essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
    return essay_df

def compute_paragraph_aggregations(paragraph_df):
    paragraph_agg_df = paragraph_df[['id','paragraph_len', 'paragraph_sent_count', 'paragraph_word_count']].groupby(['id']).agg(self.aggregations)
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    return paragraph_agg_df

In [23]:
# nth percentile function for agg
class Preprocessor:
    def __init__(self, seed):
        self.seed = seed
        
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes_dict = {
            'q': 'q', 
            ' ': 'space', 
            'NoChange': 'NoChange', 
            '.': 'full_stop', 
            ',': 'comma', 
            '\n': 'newline', 
            "'": 'single_quote', 
            '"': 'double_quote', 
            '-': 'dash', 
            '?': 'question_mark', 
            ';': 'semicolon', 
            '=': 'equals', 
            '/': 'slash', 
            '\\': 'double_backslash', 
            ':': 'colon'
        }
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 70, 100]
        self.percentiles = [5, 10, 25, 50, 75, 90, 95]
        self.percentiles_cols = [percentile(n) for n in self.percentiles]
        self.aggregations = ['mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']
        self.idf = defaultdict(float)
        
        self.essay_constructor = EssayConstructor()
    
    def get_essay_aggregations(self, essay_df):
        cols_to_drop = ['essay']
        # Total essay length
        essay_df['essay_len'] = essay_df['essay'].apply(lambda x: len(x))
        essay_df = essay_df.drop(columns=cols_to_drop)
        return essay_df
    
    def split_essays_into_words(self, essay_df):
        essay_df['word'] = essay_df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
        essay_df = essay_df.explode('word')
        # Word length (number of characters in word)
        essay_df['word_len'] = essay_df['word'].apply(lambda x: len(x))
        essay_df = essay_df[essay_df['word_len'] != 0]
        return essay_df
    
    def compute_word_aggregations(self, word_df):
        word_agg_df = word_df[['id','word_len']].groupby(['id']).agg(self.aggregations)
        word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
        word_agg_df['id'] = word_agg_df.index
        # New features: computing the # of words whose length exceed word_l
        for word_l in [5, 6, 7, 8, 9, 10, 11, 12]:
            word_agg_df[f'word_len_ge_{word_l}_count'] = word_df[word_df['word_len'] >= word_l].groupby(['id']).count().iloc[:, 0]
            word_agg_df[f'word_len_ge_{word_l}_count'] = word_agg_df[f'word_len_ge_{word_l}_count'].fillna(0)
        word_agg_df = word_agg_df.reset_index(drop=True)
        return word_agg_df
    
    def split_essays_into_sentences(self, essay_df):
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')
        essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
        # Number of characters in sentences
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
        # Number of words in sentences
        essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
        essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
        return essay_df

    def compute_sentence_aggregations(self, sent_df):
        sent_agg_df = sent_df[['id','sent_len','sent_word_count']].groupby(['id']).agg(self.aggregations)
        sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
        sent_agg_df['id'] = sent_agg_df.index
        # New features: computing the # of sentences whose (character) length exceed sent_l
        for sent_l in [50, 60, 75, 100]:
            sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_df[sent_df['sent_len'] >= sent_l].groupby(['id']).count().iloc[:, 0]
            sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_agg_df[f'sent_len_ge_{sent_l}_count'].fillna(0)
        sent_agg_df = sent_agg_df.reset_index(drop=True)
        return sent_agg_df

    def split_essays_into_paragraphs(self, essay_df):
        essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
        essay_df = essay_df.explode('paragraph')
        # Number of characters in paragraphs
        essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
        # Number of sentences in paragraphs
        essay_df['paragraph_sent_count'] = essay_df['paragraph'].apply(lambda x: len(x.split('\\.|\\?|\\!')))
        # Number of words in paragraphs
        essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
        essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
        return essay_df

    def compute_paragraph_aggregations(self, paragraph_df):
        paragraph_agg_df = paragraph_df[['id','paragraph_len', 'paragraph_sent_count', 'paragraph_word_count']].groupby(['id']).agg(self.aggregations)
        paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
        paragraph_agg_df['id'] = paragraph_agg_df.index
        paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
        return paragraph_agg_df
        
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            # make dictionary entry for "move from X to Y"
            di["move_to"] = 0
            
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
                else:
                    # we can do this because there are no missing values
                    di["move_to"] += v
            ret.append(di)
        
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'activity_{act}_tfidf_count' for act in ret.columns]
        normal_cols = [f'activity_{act}_normal_count' for act in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg

    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
            
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'{colname}_{event}_tfidf_count' for event in ret.columns]
        normal_cols = [f'{colname}_{event}_normal_count' for event in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg

    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes_dict.keys():
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
            
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'text_change_{self.text_changes_dict[txt_change]}_tfidf_count' for txt_change in ret.columns]
        normal_cols = [f'text_change_{self.text_changes_dict[txt_change]}_normal_count' for txt_change in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg
    
    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret

    # Credit: https://www.kaggle.com/code/abdullahmeda/enter-ing-the-timeseries-space-sec-3-new-aggs/notebook
    def make_space_features(self, df):
        df['up_time_lagged'] = df.groupby('id')['up_time'].shift(1).fillna(df['down_time'])
        df['time_diff'] = abs(df['down_time'] - df['up_time_lagged']) / 1000

        group = df.groupby('id')['time_diff']
        largest_lantency = group.max()
        smallest_lantency = group.min()
        median_lantency = group.median()
        initial_pause = df.groupby('id')['down_time'].first() / 1000
        pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
        pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
        pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
        pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
        pauses_3_sec = group.apply(lambda x: (x > 3).sum())
        
        result = pd.DataFrame({
            'id': df['id'].unique(),
            'largest_lantency': largest_lantency,
            'smallest_lantency': smallest_lantency,
            'median_lantency': median_lantency,
            'initial_pause': initial_pause,
            'pauses_half_sec': pauses_half_sec,
            'pauses_1_sec': pauses_1_sec,
            'pauses_1_half_sec': pauses_1_half_sec,
            'pauses_2_sec': pauses_2_sec,
            'pauses_3_sec': pauses_3_sec,
        }).reset_index(drop=True)
        return result
    
    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        for percentile in self.percentiles:
            tmp_df[f'input_word_length_pct_{percentile}'] = tmp_df['text_change'].apply(lambda x: np.percentile([len(i) for i in x] if len(x) > 0 else 0, 
                                                                                                               percentile))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def make_feats(self, df: pd.DataFrame, save_essays_path: str):
        
        print("Starting to engineer features")
        
        # initialize features dataframe
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        # get essay feats
        print("Getting essays")
        essay_df = self.essay_constructor.getEssays(df)
        essay_df.to_csv(save_essays_path, index=False)

        print("Getting essay aggregations data")
        essay_agg_df = self.get_essay_aggregations(essay_df)
        feats = feats.merge(essay_agg_df, on='id', how='left')

        print("Getting essay word aggregations data")
        word_df = self.split_essays_into_words(essay_df)
        word_agg_df = self.compute_word_aggregations(word_df)
        feats = feats.merge(word_agg_df, on='id', how='left')

        print("Getting essay sentence aggregations data")
        sent_df = self.split_essays_into_sentences(essay_df)
        sent_agg_df = self.compute_sentence_aggregations(sent_df)
        feats = feats.merge(sent_agg_df, on='id', how='left')

        print("Getting essay paragraph aggregations data")
        paragraph_df = self.split_essays_into_paragraphs(essay_df)
        paragraph_agg_df = self.compute_paragraph_aggregations(paragraph_df)
        feats = feats.merge(paragraph_agg_df, on='id', how='left')
        
        # engineer counts data
        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        # space features
        print("Engineering space-related data")
        tmp_df = self.make_space_features(df)
        feats = feats.merge(tmp_df, on='id', how='left')
        
        # get shifted features
        # time shift
        print("Engineering time data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)
        
        # cursor position shift
        print("Engineering cursor position data - gaps")
        for gap in self.gaps: 
            print(f"> for gap {gap}")
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)
        
        # word count shift
        print("Engineering word count data - gaps")
        for gap in self.gaps: 
            print(f"> for gap {gap}")
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        # get aggregate statistical features
        print("Engineering statistical summaries for features")
        # [(feature name, [ stat summaries to add ])]
        percentiles_cols = [percentile(n) for n in self.percentiles]
        feats_stat = [
            ('action_time',self.percentiles_cols),
            ('activity', ['nunique']),
            ('down_event', [ 'nunique']),
            ('up_event', [ 'nunique']),
            ('text_change', [ 'nunique']),
            ] 

        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['first','last', 'max', 'min', 'mean', 'std', 'sem', 'skew', pd.DataFrame.kurt]+ percentiles_cols),
                (f'cursor_position_change{gap}', ['first','last','max', 'mean', 'std','sem', 'skew', pd.DataFrame.kurt]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'sum', 'sem', 'skew', pd.DataFrame.kurt] + percentiles_cols),
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                    
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left') 

        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        # compare feats
#         print("Engineering ratios data")
#         feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
#         feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
#         feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
#         feats['idle_time_ratio'] = feats['action_time_gap1_mean'] / feats['up_time_max']
        
        print("Done!")
        return feats

In [24]:
#bruteforce agg
train_agg_fe_df = train_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(
    ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
train_agg_fe_df = train_agg_fe_df.add_prefix("tmp_")
train_agg_fe_df.reset_index(inplace=True)

test_agg_fe_df = test_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(
    ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix("tmp_")
test_agg_fe_df.reset_index(inplace=True)

In [25]:
def duration_features(train_logs,test_logs):
    tr_logs,te_logs = copy.deepcopy(train_logs),copy.deepcopy(test_logs)
    data = []
    for logs in [tr_logs, te_logs]:
        #logs['up_time_lagged'] = logs.groupby('id')['up_time'].shift(1).fillna(logs['down_time'])
        #logs['time_diff'] = abs(logs['down_time'] - logs['up_time_lagged']) / 1000
        logs['time_diff'] = logs.groupby('id')['down_time'].diff()
        group = logs.groupby('id')['time_diff']
        largest_lantency = group.max()
        smallest_lantency = group.min()
        median_lantency = group.median()
        initial_pause = logs.groupby('id')['down_time'].first() / 1000
        pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
        pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
        pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
        pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
        pauses_3_sec = group.apply(lambda x: (x > 3).sum())
        data.append(pd.DataFrame({
            'id': logs['id'].unique(),
            'largest_lantency': largest_lantency,
            'smallest_lantency': smallest_lantency,
            'median_lantency': median_lantency,
            'initial_pause': initial_pause,
            'pauses_half_sec': pauses_half_sec,
            'pauses_1_sec': pauses_1_sec,
            'pauses_1_half_sec': pauses_1_half_sec,
            'pauses_2_sec': pauses_2_sec,
            'pauses_3_sec': pauses_3_sec,
        }).reset_index(drop=True))
    return data

In [26]:
def essay_CountVectorizer(train_essays,test_essays):
    train_essaysdf = copy.deepcopy(train_essays['essay'])
    test_essaysdf = copy.deepcopy(test_essays['essay'])
    train_essaysdf = pd.DataFrame({'id': train_essaysdf.index, 'essay': train_essaysdf.values})
    test_essaysdf = pd.DataFrame({'id': test_essaysdf.index, 'essay': test_essaysdf.values})
    merged_data = train_essaysdf.merge(train_scores, on='id')

    count_vectorizer = CountVectorizer(ngram_range=(1, 2))
    X_tokenizer_train = count_vectorizer.fit_transform(merged_data['essay'])
    X_tokenizer_test = count_vectorizer.transform(test_essaysdf['essay'])
    y = merged_data['score']

    X_tokenizer_train = X_tokenizer_train.todense()
    X_tokenizer_test = X_tokenizer_test.todense()

    train_count_vector = pd.DataFrame()
    test_count_vector = pd.DataFrame()
    for i in range(X_tokenizer_train.shape[1]) : 
        L = list(X_tokenizer_train[:,i])
        li = [int(x) for x in L ]
        train_count_vector[f'feature {i}'] = li
    for i in range(X_tokenizer_test.shape[1]): 
        L = list(X_tokenizer_test[:,i])
        li = [int(x) for x in L ]
        test_count_vector[f'feature {i}'] = li

    df_train_index = train_essaysdf['id']
    df_test_index = test_essaysdf['id']
    train_count_vector.loc[:, 'id'] = df_train_index
    test_count_vector.loc[:, 'id'] = df_test_index

    #保留部分特征，保留90%以上的人出现过的特征
    save_cols = []
    for i in train_count_vector.columns:
        if sum(train_count_vector[i]==0)/len(train_count_vector)<0.1:
            save_cols.append(i)
    return train_count_vector,test_count_vector,save_cols

In [27]:
%%time
def other_features(df,method='train'):
    a = pd.DataFrame()
    a['Input_all_ratio'] = df.groupby(['id']).apply(lambda x:sum(x['activity']!='Input'))/df.groupby(['id']).apply(lambda x:sum(x['activity']=='Input'))
    a['all_q_ratio'] = df.groupby(['id']).apply(lambda x:sum(x['down_event']!='q'))/df.groupby(['id']).apply(lambda x:sum(x['down_event']=='q'))
    activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
    events_dict = {
                    'q':'q', 
                    'Space':'Space', 
                    'Backspace':'Backspace', 
                    'Shift':'Shift', 
                    'ArrowRight':'ArrowRight', 
                    'Leftclick':'Leftclick', 
                    'ArrowLeft':'ArrowLeft', 
                    '.':'fullstop', 
                    ',':'comma', 
                    'ArrowDown':'ArrowDown', 
                    'ArrowUp':'ArrowUp', 
                    'Enter':'Enter', 
                    'CapsLock':'CapsLock', 
                    "'":'single_quote', 
                    'Delete':'Delete', 
                    'Unidentified':'Unidentified',
                  }
    for i in tqdm(activities):
        for j in events_dict:
            a[f'{i}_{events_dict[j]}_count'] = df.groupby('id').apply(lambda x:len(x[(x['activity']==i)&(x['down_event']==j)]))
    return a.reset_index()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [28]:
def language_error(df,df_essays):
    a = pd.DataFrame()
    df['down_event_shift'] = df.groupby('id')['down_event'].shift(-1)
    letter_upper = df.groupby('id').apply(lambda x:len(x[(x['down_event']=='CapsLock')|((x['down_event']=='Shift')&(x['down_event_shift']=='q'))]))
    a['letter_big_count'] = letter_upper.values
    a['id'] = df['id'].unique()
    
    essay_df = copy.deepcopy(df_essays)
    essay_df['id'] = essay_df.index
    
    #避免将qqq.).切分成多个句子
    #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))
    
    essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())    
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    essay_df = essay_df[essay_df['sent_len']!=0]
    errors_num = (essay_df.groupby('id').apply(len)-letter_upper).values
    a['error_num'] = errors_num                          #如果句子个数大于大写字母按键次数，那么文章会有语法错误
    return a
def sentence_error(df):
    essay_df = copy.deepcopy(df)
    essay_df['id'] = essay_df.index
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    essay_df = essay_df[essay_df['paragraph_len']!=0]
    essay_df['only_space'] = essay_df['paragraph'].apply(lambda x:'q' not in x)
    essay_df = essay_df[essay_df['only_space']==False]
    a = pd.DataFrame()
    a['para_error'] = essay_df.groupby('id').apply(lambda x:len(x[x['paragraph_len']<25]))   #一个段落字符过少可能不是完整的一句话，可能存在语法错误

    return a.reset_index()

def language_error_letter(df):
    essay_df = copy.deepcopy(df)
    essay_df['id'] = essay_df.index
    
    #避免将qqq.).切分成多个句子
    #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))
    
    essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip()) 
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
    essay_df['language_error_letter'] = essay_df['sent'].apply(lambda x:x[0])
    essay_df['if_q'] = essay_df['language_error_letter'].apply(lambda x:x.lower()=='q')
    essay_df = essay_df[essay_df['if_q']==True]
    a = pd.DataFrame()
    a['language_error_letter'] = essay_df.groupby('id').apply(lambda x:len(x[x['language_error_letter']=='q']))
    return a.reset_index()

In [29]:
use_pre_fe_data = False
if use_pre_fe_data:
    pass
else:
    preprocessor_train = Preprocessor(seed = 42)
    train_feats = preprocessor_train.make_feats(train_logs,save_essays_path = 'train_essays.csv')
    del preprocessor_train
    gc.collect()

preprocessor_test = Preprocessor(seed = 42)
test_feats = preprocessor_test.make_feats(test_logs,save_essays_path = 'test_essays.csv')
print('<train_feats and test_feats done.>')

train_feats = train_feats.merge(train_agg_fe_df, on='id', how='left')
test_feats = test_feats.merge(test_agg_fe_df, on='id', how='left')
print('<merge train and test agg done.>')

train_eD592674, test_eD592674 = duration_features(train_logs,test_logs)
train_feats = train_feats.merge(train_eD592674, on='id', how='left')
test_feats = test_feats.merge(test_eD592674, on='id', how='left')
train_feats = train_feats.merge(train_scores, on='id', how='left')
print('<merge duration features|train label done.>')

test_essays = (EssayConstructor().getEssays(test_logs))
test_essays.index = test_essays['id'].values
test_essays.drop(columns='id',inplace=True)
train_count_vector,test_count_vector,save_cols = essay_CountVectorizer(train_essays,test_essays)
train_feats = train_feats.merge(train_count_vector[save_cols], on='id', how='left')
test_feats = test_feats.merge(test_count_vector[save_cols], on='id', how='left')
print('<merge train and test essay CountVectorizer done.>')

if os.path.exists('/kaggle/input/lgbm-and-nn-on-sentences'):  
    train_agg_ratio = pd.read_csv('/kaggle/input/lgbm-and-nn-on-sentences/train_agg_ratio.csv')
else:
    train_agg_ratio = other_features(train_logs)
#删除掉全部为0的列
save_cols = []
for i in train_agg_ratio.columns:
    if sum(train_agg_ratio[i]==0)<len(train_agg_ratio):
        save_cols.append(i)
test_agg_ratio = other_features(test_logs)                          
train_feats = train_feats.merge(train_agg_ratio[save_cols], on='id', how='left')
test_feats = test_feats.merge(test_agg_ratio[save_cols], on='id', how='left')
train_agg_ratio.to_csv('/kaggle/working/train_agg_ratio.csv',index=0)
print('<merge other featuers done.>')

tr_language_error_agg = language_error(train_logs,train_essays)
te_language_error_agg = language_error(test_logs,test_essays)
train_feats = train_feats.merge(tr_language_error_agg, on='id', how='left')
test_feats = test_feats.merge(te_language_error_agg, on='id', how='left')

tr_sentence_error_agg = sentence_error(train_essays)
te_sentence_error_agg = sentence_error(test_essays)
train_feats = train_feats.merge(tr_sentence_error_agg, on='id', how='left')
test_feats = test_feats.merge(te_sentence_error_agg, on='id', how='left')

tr_language_error_letter_agg =  language_error_letter(train_essays_with_upper)
test_essays_with_upper = getEssays_with_upper(test_logs)
te_language_error_letter_agg =  language_error_letter(test_essays_with_upper)
train_feats = train_feats.merge(tr_language_error_letter_agg, on='id', how='left')
test_feats = test_feats.merge(te_language_error_letter_agg, on='id', how='left')
print('<merge language errors done.>')

Starting to engineer features
Getting essays


100%|██████████| 2471/2471 [00:16<00:00, 149.46it/s]


Getting essay aggregations data
Getting essay word aggregations data
Getting essay sentence aggregations data
Getting essay paragraph aggregations data
Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 5534.12it/s]


Engineering event counts data


...

<merge train and test essay CountVectorizer done.>


100%|██████████| 5/5 [00:00<00:00,  8.17it/s]


<merge other featuers done.>


100%|██████████| 3/3 [00:00<00:00, 558.97it/s]

<merge language errors done.>


In [30]:
target_col = ['score']
drop_cols = ['id']
has_inf_cols = ['activity_Remove/Cut_tfidf_count',
 'activity_Replace_tfidf_count',
 'activity_Paste_tfidf_count',
 'activity_move_to_tfidf_count',
 'down_event_Backspace_tfidf_count',
 'down_event_Shift_tfidf_count',
 'down_event_ArrowRight_tfidf_count',
 'down_event_Leftclick_tfidf_count',
 'down_event_ArrowLeft_tfidf_count',
 'down_event_._tfidf_count',
 'down_event_,_tfidf_count',
 'down_event_ArrowDown_tfidf_count',
 'down_event_ArrowUp_tfidf_count',
 'down_event_Enter_tfidf_count',
 'down_event_CapsLock_tfidf_count',
 "down_event_'_tfidf_count",
 'down_event_Delete_tfidf_count',
 'down_event_Unidentified_tfidf_count',
 'up_event_Backspace_tfidf_count',
 'up_event_Shift_tfidf_count',
 'up_event_ArrowRight_tfidf_count',
 'up_event_Leftclick_tfidf_count',
 'up_event_ArrowLeft_tfidf_count',
 'up_event_._tfidf_count',
 'up_event_,_tfidf_count',
 'up_event_ArrowDown_tfidf_count',
 'up_event_ArrowUp_tfidf_count',
 'up_event_Enter_tfidf_count',
 'up_event_CapsLock_tfidf_count',
 "up_event_'_tfidf_count",
 'up_event_Delete_tfidf_count',
 'up_event_Unidentified_tfidf_count',
 'text_change_comma_tfidf_count',
 'text_change_newline_tfidf_count',
 'text_change_single_quote_tfidf_count',
 'text_change_double_quote_tfidf_count',
 'text_change_dash_tfidf_count',
 'text_change_question_mark_tfidf_count',
 'text_change_semicolon_tfidf_count',
 'text_change_equals_tfidf_count',
 'text_change_slash_tfidf_count',
 'text_change_double_backslash_tfidf_count',
 'text_change_colon_tfidf_count']
drop_cols_NN = []
for i in train_feats.columns:
    if 'action_time' in i:
        drop_cols_NN.append(i)
    if 'up_time' in i:
        drop_cols_NN.append(i)
cols = [col for col in train_feats.columns if col not in drop_cols+drop_cols_NN+has_inf_cols]  

In [31]:
TASK = 'reg'
task = Task(TASK)
TIMEOUT = 10000
N_THREADS = 2
TRAIN_BS = 128
USE_PLR = True
USE_QNT = True
N_FOLDS = 5
RANDOM_STATE = 42
ADVANCED_ROLES = False
TARGET_NAME = 'score'
roles = {'target':TARGET_NAME}

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [32]:
def score(task, y_true, y_pred):
    if task.name == 'binary':
        return roc_auc_score(y_true, y_pred)
    elif task.name == 'multiclass':
        return log_loss(y_true, y_pred)
    elif task.name == 'reg' or task.name == 'multi:reg':
        return mean_squared_error(y_true,y_pred,squared=False)
    else:
        raise 'Task is not correct.'
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"
def take_pred_from_task(pred, task):
    if task.name == 'binary' or task.name == 'reg':
        return pred[:, 0]
    elif task.name == 'multiclass' or task.name == 'multi:reg':
        return pred
    else:
        raise 'Task is not correct.'

In [33]:
OOF_PREDS_mlp = np.zeros(len(train_feats))
algo = "mlp"
for RANDOM_STATE in range(42,43):
    automl = TabularAutoML(
        task = task, 
        timeout = TIMEOUT,
        cpu_limit = N_THREADS,
        general_params = {"use_algos": [[algo]]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
        nn_params = {
            "n_epochs": 10, 
            "bs": TRAIN_BS, 
            "num_workers": 0, 
            "path_to_save": None, 
            "freeze_defaults": True, 
            "cont_embedder": use_plr(USE_PLR), 
            "hidden_size": 32,
            'clip_grad': True, 
            'clip_grad_params': {'max_norm': 1},
        },
        nn_pipeline_params = {"use_qnt": USE_QNT, "use_te": False},
        reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE, 'advanced_roles': ADVANCED_ROLES},
    )
    OOF_PREDS_mlp += take_pred_from_task((automl.fit_predict(train_data=train_feats[cols],roles = {'target':'score'}, verbose = 3)).data, task)  /len(range(42,43))
    joblib.dump(automl,f'MLP_{RANDOM_STATE}.pkl')

[09:03:36] Stdout logging level is INFO3.
[09:03:36] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[09:03:36] Task: reg

[09:03:36] Start automl preset with listed constraints:
[09:03:36] - time: 10000.00 seconds
[09:03:36] - CPU: 2 cores
[09:03:36] - memory: 16 GB

[09:03:36] Train data shape: (2471, 498)

[09:03:37] Layer 1 train process start. Time left 9999.55 secs
[09:03:38] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 ...
[09:03:38] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 =====
[09:03:43] Epoch: 0, train loss: 0.9810305833816528, val loss: 0.7626069784164429, val metric: -0.7649670243263245
[09:03:48] Epoch: 1, train loss: 0.6814952492713928, val loss: 0.4305188059806824, val metric: -0.4298667907714844
[09:03:53] Epoch: 2, train loss: 0.45020735263824463, val loss: 0.32762372493743896, val metric: -0.3262951672077179
[09:03:57] Epoch: 3, train loss: 0.397036612033844, val loss: 0.3089464008808136

In [34]:
metrics.mean_squared_error(train_feats['score'],OOF_PREDS_mlp,squared=False)

0.5966808076245195

In [35]:
test_cols = [col for col in train_feats.columns if col not in drop_cols+drop_cols_NN+target_col]

In [36]:
TEST_PREDS_mlp = np.zeros(len(test_feats))
for i in range(42,43):
    automl = joblib.load('/kaggle/working/MLP_{}.pkl'.format(i))
    TEST_PREDS_mlp += automl.predict(test_feats[test_cols]).data[:, 0] / len(range(42,43))

# 0.582 public notebook

In [37]:
import polars as pl
import pandas as pd
import numpy as np
import re
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings("ignore")

In [38]:
num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def count_by_values(df, colname, values):
    fts = df.select(pl.col('id').unique(maintain_order=True))
    for i, value in enumerate(values):
        tmp_df = df.group_by('id').agg(pl.col(colname).is_in([value]).sum().alias(f'{colname}_{i}_cnt'))
        fts  = fts.join(tmp_df, on='id', how='left') 
    return fts


def dev_feats(df):
    
    print("< Count by values features >")
    
    feats = count_by_values(df, 'activity', activities)
    feats = feats.join(count_by_values(df, 'text_change', text_changes), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'down_event', events), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'up_event', events), on='id', how='left') 

    print("< Input words stats features >")

    temp = df.filter((~pl.col('text_change').str.contains('=>')) & (pl.col('text_change') != 'NoChange'))
    temp = temp.group_by('id').agg(pl.col('text_change').str.concat('').str.extract_all(r'q+'))
    temp = temp.with_columns(input_word_count = pl.col('text_change').list.lengths(),
                             input_word_length_mean = pl.col('text_change').apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_max = pl.col('text_change').apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_std = pl.col('text_change').apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_median = pl.col('text_change').apply(lambda x: np.median([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_skew = pl.col('text_change').apply(lambda x: skew([len(i) for i in x] if len(x) > 0 else 0)))
    temp = temp.drop('text_change')
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Numerical columns features >")

    temp = df.group_by("id").agg(pl.sum('action_time').suffix('_sum'), pl.mean(num_cols).suffix('_mean'), pl.std(num_cols).suffix('_std'),
                                 pl.median(num_cols).suffix('_median'), pl.min(num_cols).suffix('_min'), pl.max(num_cols).suffix('_max'),
                                 pl.quantile(num_cols, 0.5).suffix('_quantile'))
    feats = feats.join(temp, on='id', how='left') 


    print("< Categorical columns features >")
    
    temp  = df.group_by("id").agg(pl.n_unique(['activity', 'down_event', 'up_event', 'text_change']))
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Idle time features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.group_by("id").agg(inter_key_largest_lantency = pl.max('time_diff'),
                                   inter_key_median_lantency = pl.median('time_diff'),
                                   mean_pause_time = pl.mean('time_diff'),
                                   std_pause_time = pl.std('time_diff'),
                                   total_pause_time = pl.sum('time_diff'),
                                   pauses_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 0.5) & (pl.col('time_diff') < 1)).count(),
                                   pauses_1_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1) & (pl.col('time_diff') < 1.5)).count(),
                                   pauses_1_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1.5) & (pl.col('time_diff') < 2)).count(),
                                   pauses_2_sec = pl.col('time_diff').filter((pl.col('time_diff') > 2) & (pl.col('time_diff') < 3)).count(),
                                   pauses_3_sec = pl.col('time_diff').filter(pl.col('time_diff') > 3).count(),)
    feats = feats.join(temp, on='id', how='left') 
    
    print("< P-bursts features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('time_diff')<2)
    temp = temp.with_columns(pl.when(pl.col("time_diff") & pl.col("time_diff").is_last()).then(pl.count()).over(pl.col("time_diff").rle_id()).alias('P-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('P-bursts').suffix('_mean'), pl.std('P-bursts').suffix('_std'), pl.count('P-bursts').suffix('_count'),
                                   pl.median('P-bursts').suffix('_median'), pl.max('P-bursts').suffix('_max'),
                                   pl.first('P-bursts').suffix('_first'), pl.last('P-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left') 


    print("< R-bursts features >")

    temp = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('activity').is_in(['Remove/Cut']))
    temp = temp.with_columns(pl.when(pl.col("activity") & pl.col("activity").is_last()).then(pl.count()).over(pl.col("activity").rle_id()).alias('R-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('R-bursts').suffix('_mean'), pl.std('R-bursts').suffix('_std'), 
                                   pl.median('R-bursts').suffix('_median'), pl.max('R-bursts').suffix('_max'),
                                   pl.first('R-bursts').suffix('_first'), pl.last('R-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left')
    
    return feats


def train_valid_split(data_x, data_y, train_idx, valid_idx):
    x_train = data_x.iloc[train_idx]
    y_train = data_y[train_idx]
    x_valid = data_x.iloc[valid_idx]
    y_valid = data_y[valid_idx]
    return x_train, y_train, x_valid, y_valid


# def evaluate(data_x, data_y, model, random_state=42, n_splits=5, test_x=None):
#     skf    = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
#     test_y = np.zeros(len(data_x)) if (test_x is None) else np.zeros((len(test_x), n_splits))
#     for i, (train_index, valid_index) in enumerate(skf.split(data_x, data_y.astype(str))):
#         train_x, train_y, valid_x, valid_y = train_valid_split(data_x, data_y, train_index, valid_index)
#         model.fit(train_x, train_y)
#         if test_x is None:
#             test_y[valid_index] = model.predict(valid_x)
#         else:
#             test_y[:, i] = model.predict(test_x)
#     return test_y if (test_x is None) else np.mean(test_y, axis=1)


def evaluate(data_x, data_y, model, random_state=42, n_splits=5, test_x=None):
    skf    = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    test_y = np.zeros(len(data_x)) 
    p_y   = np.zeros((len(test_x), n_splits))
    for i, (train_index, valid_index) in enumerate(skf.split(data_x, data_y.astype(str))):
        train_x, train_y, valid_x, valid_y = train_valid_split(data_x, data_y, train_index, valid_index)
        model.fit(train_x, train_y)
        test_y[valid_index] = model.predict(valid_x)
        p_y[:, i] = model.predict(test_x)

    return test_y, np.mean(p_y, axis=1)


In [39]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

AGGREGATIONS = ['count', 'mean', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']

def reconstruct_essay(currTextInput):
    essayText = ""
    for Input in currTextInput.values:
        if Input[0] == 'Replace':
            replaceTxt = Input[2].split(' => ')
            essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
            continue
        if Input[0] == 'Paste':
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
            continue
        if Input[0] == 'Remove/Cut':
            essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
            continue
        if "M" in Input[0]:
            croppedTxt = Input[0][10:]
            splitTxt = croppedTxt.split(' To ')
            valueArr = [item.split(', ') for item in splitTxt]
            moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
            if moveData[0] != moveData[2]:
                if moveData[0] < moveData[2]:
                    essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                else:
                    essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
            continue
        essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
    return essayText


def get_essay_df(df):
    df       = df[df.activity != 'Nonproduction']
    temp     = df.groupby('id').apply(lambda x: reconstruct_essay(x[['activity', 'cursor_position', 'text_change']]))
    essay_df = pd.DataFrame({'id': df['id'].unique().tolist()})
    essay_df = essay_df.merge(temp.rename('essay'), on='id')
    return essay_df


def word_feats(df):
    essay_df = df
    df['word'] = df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    df = df.explode('word')
    df['word_len'] = df['word'].apply(lambda x: len(x))
    df = df[df['word_len'] != 0]

    word_agg_df = df[['id','word_len']].groupby(['id']).agg(AGGREGATIONS)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df


def sent_feats(df):
    df['sent'] = df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    df = df.explode('sent')
    df['sent'] = df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    df['sent_len'] = df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    df['sent_word_count'] = df['sent'].apply(lambda x: len(x.split(' ')))

    sp_df = df[df.sent_len==0].reset_index(drop=True)
    sp_df = sp_df.groupby(['id'])['essay'].count().reset_index()
    sp_df.columns = ['id', 'sp_cn']
    df = df[df.sent_len!=0].reset_index(drop=True)

    def sent_mic_feat(x):
        res = []
        numx = []
        for h in x.split(','):
            numx.append(len(h.split( )))
        res = [np.mean(numx), np.std(numx), np.max(numx), np.min(numx), len(numx)]
        return res
    df['mic_feat'] = df['sent'].apply(sent_mic_feat)
    df = pd.concat([df,df['mic_feat'].apply(pd.Series)], axis=1).drop(['mic_feat'], axis=1)
    df.columns = ['id','essay','word','sent','sent_len','sent_word_count','mic_f_mean','mic_f_std','mic_f_max','mic_f_min','mic_f_cn']

    sent_agg_df = pd.concat([df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), 
                                df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS), 
                                df[['id','mic_f_mean']].groupby(['id']).agg(AGGREGATIONS), 
                                df[['id','mic_f_std']].groupby(['id']).agg(AGGREGATIONS), 
                                df[['id','mic_f_max']].groupby(['id']).agg(AGGREGATIONS), 
                                df[['id','mic_f_min']].groupby(['id']).agg(AGGREGATIONS), 
                                df[['id','mic_f_cn']].groupby(['id']).agg(AGGREGATIONS)], 
                                axis=1)
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    sent_agg_df = sent_agg_df.merge(sp_df, on=['id'], how='left')
    return sent_agg_df


def parag_feats(df):
    df['paragraph'] = df['essay'].apply(lambda x: x.split('\n'))
    df = df.explode('paragraph')
    # Number of characters in paragraphs
    df['paragraph_len'] = df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    df['paragraph_word_count'] = df['paragraph'].apply(lambda x: len(x.split(' ')))
    df = df[df.paragraph_len!=0].reset_index(drop=True)
    
    paragraph_agg_df = pd.concat([df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), 
                                  df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

def product_to_keys(logs, essays):
    essays['product_len'] = essays.essay.str.len()
    tmp_df = logs[logs.activity.isin(['Input', 'Remove/Cut'])].groupby(['id']).agg({'activity': 'count'}).reset_index().rename(columns={'activity': 'keys_pressed'})
    essays = essays.merge(tmp_df, on='id', how='left')
    essays['product_to_keys'] = essays['product_len'] / essays['keys_pressed']
    return essays[['id', 'product_to_keys']]

def get_keys_pressed_per_second(logs):
    temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
    temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
    temp_df = temp_df.merge(temp_df_2, on='id', how='left')
    temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
    return temp_df[['id', 'keys_per_second']]


In [40]:
data_path     = '/kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(data_path + 'train_logs.csv')
train_feats   = dev_feats(train_logs)
train_feats   = train_feats.collect().to_pandas()

print('< Essay Reconstruction >')
train_logs             = train_logs.collect().to_pandas()
train_essays           = get_essay_df(train_logs)
train_feats            = train_feats.merge(word_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(sent_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(parag_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(get_keys_pressed_per_second(train_logs), on='id', how='left')
train_feats            = train_feats.merge(product_to_keys(train_logs, train_essays), on='id', how='left')


print('< Mapping >')
train_scores   = pd.read_csv(data_path + 'train_scores.csv')
data           = train_feats.merge(train_scores, on='id', how='left')
x              = data.drop(['id', 'score'], axis=1)
y              = data['score'].values
print(f'Number of features: {len(x.columns)}')


print('< Testing Data >')
test_logs   = pl.scan_csv(data_path + 'test_logs.csv')
test_feats  = dev_feats(test_logs)
test_feats  = test_feats.collect().to_pandas()

test_logs             = test_logs.collect().to_pandas()
test_essays           = get_essay_df(test_logs)
test_feats            = test_feats.merge(word_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(get_keys_pressed_per_second(test_logs), on='id', how='left')
test_feats            = test_feats.merge(product_to_keys(test_logs, test_essays), on='id', how='left')


test_ids = test_feats['id'].values
testin_x = test_feats.drop(['id'], axis=1)

< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >
< Essay Reconstruction >
< Mapping >
Number of features: 216
< Testing Data >
< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >


In [41]:
print('< Learning and Evaluation >')
param_best = {'reg_alpha': 0.5220090593805945, 'reg_lambda': 8.852759022621703, 'colsample_bytree': 0.5401337103950415, 'subsample': 0.8100111797463894, 'learning_rate': 0.012550862839485352, 'max_depth': 20, 'num_leaves': 11, 'min_child_samples': 10, 'bagging_fraction': 0.8719524472246258, 'lambda_l1': 0.030531675309677896, 'lambda_l2': 0.0004004028086907504, 'bagging_freq': 1}
param = {'n_estimators': 1024,
         'metric': 'rmse',
         'random_state': 42,
         'force_col_wise': True,
         'verbosity': 1,
         **param_best
}

solution = LGBMRegressor(**param)
lx_off_pred1, lx_y_pred1   = evaluate(x.copy(), y.copy(), solution, test_x=testin_x.copy())

< Learning and Evaluation >
[LightGBM] [Warning] lambda_l1 is set=0.030531675309677896, reg_alpha=0.5220090593805945 will be ignored. Current value: lambda_l1=0.030531675309677896
[LightGBM] [Warning] bagging_fraction is set=0.8719524472246258, subsample=0.8100111797463894 will be ignored. Current value: bagging_fraction=0.8719524472246258
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=0.0004004028086907504, reg_lambda=8.852759022621703 will be ignored. Current value: lambda_l2=0.0004004028086907504
[LightGBM] [Warning] lambda_l1 is set=0.030531675309677896, reg_alpha=0.5220090593805945 will be ignored. Current value: lambda_l1=0.030531675309677896
[LightGBM] [Warning] bagging_fraction is set=0.8719524472246258, subsample=0.8100111797463894 will be ignored. Current value: bagging_fraction=0.8719524472246258
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Curr

In [42]:
from tqdm.auto import tqdm

tokenized_texts_train = []
for text in tqdm(train_essays['word'].tolist()):
    # tokenized_texts_train.append(tokenizer.tokenize(text))
    tokenized_texts_train.append(text)

tokenized_texts_test = []
for text in tqdm(test_essays['word'].tolist()):
    # tokenized_texts_test.append(tokenizer.tokenize(text))
    tokenized_texts_test.append(text)

text_tfidf = tokenized_texts_train+tokenized_texts_test
from sklearn.feature_extraction.text import TfidfVectorizer
def dummy(text):
    return text
vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                            max_features=150,
                            lowercase=False, 
                            sublinear_tf=True, 
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode')

vectorizer.fit(text_tfidf)
tffeature_train = vectorizer.transform(tokenized_texts_train).toarray()
tffeature_test = vectorizer.transform(tokenized_texts_test).toarray()


collst = []
for i in range(150):
    tpcol = f'tffeat_{i}'
    collst.append(tpcol)
    x[tpcol] = tffeature_train[:,i]
    testin_x[tpcol] = tffeature_test[:,i]
    data[tpcol] = tffeature_train[:,i]

  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [43]:
print('< Learning and Evaluation >')
# param_best = {'reg_alpha': 0.5220090593805945, 'reg_lambda': 8.852759022621703, 'colsample_bytree': 0.5401337103950415, 'subsample': 0.8100111797463894, 'learning_rate': 0.012550862839485352, 'max_depth': 20, 'num_leaves': 11, 'min_child_samples': 10, 'bagging_fraction': 0.8719524472246258, 'lambda_l1': 0.030531675309677896, 'lambda_l2': 0.0004004028086907504, 'bagging_freq': 1}
param_best = {'reg_alpha': 0.31392305787884783, 'reg_lambda': 0.048717707444993374, 'colsample_bytree': 0.5487892487545942, 'subsample': 0.7167488247593671, 'learning_rate': 0.0074954259728873115, 'max_depth': 20, 'num_leaves': 17, 'min_child_samples': 32, 'bagging_fraction': 0.7037344869543838, 'lambda_l1': 2.8206447468653433e-05, 'lambda_l2': 2.25550846605472e-08, 'bagging_freq': 2}
param = {'n_estimators': 1024,
         'metric': 'rmse',
         'random_state': 42,
         'force_col_wise': True,
         'verbosity': 1,
         **param_best
}

solution = LGBMRegressor(**param)
lx_off_pred2, lx_y_pred2   = evaluate(x.copy(), y.copy(), solution, test_x=testin_x.copy()) 

< Learning and Evaluation >
[LightGBM] [Warning] lambda_l1 is set=2.8206447468653433e-05, reg_alpha=0.31392305787884783 will be ignored. Current value: lambda_l1=2.8206447468653433e-05
[LightGBM] [Warning] bagging_fraction is set=0.7037344869543838, subsample=0.7167488247593671 will be ignored. Current value: bagging_fraction=0.7037344869543838
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=2.25550846605472e-08, reg_lambda=0.048717707444993374 will be ignored. Current value: lambda_l2=2.25550846605472e-08
[LightGBM] [Warning] lambda_l1 is set=2.8206447468653433e-05, reg_alpha=0.31392305787884783 will be ignored. Current value: lambda_l1=2.8206447468653433e-05
[LightGBM] [Warning] bagging_fraction is set=0.7037344869543838, subsample=0.7167488247593671 will be ignored. Current value: bagging_fraction=0.7037344869543838
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ig

In [44]:
best_sc = 1
for w in np.arange(0, 1.001, 0.001):
    sc = metrics.mean_squared_error(y, 
                                    w * lx_off_pred1 + (1-w) * lx_off_pred2, 
                                    squared=False)
    if sc < best_sc:
        best_sc = sc
        best_w = w
        
print('Composition OOF score = {:.5f}'.format(best_sc))
print('Composition best W = {:.3f}'.format(best_w))

Composition OOF score = 0.59565
Composition best W = 0.236


In [45]:
# W = [best_w, 1 - best_w]
#lx_off_pred = lx_off_pred1 * W[0] + lx_off_pred2 * W[1]
# lx_y_pred = lx_y_pred1 * W[0] + lx_y_pred2 * W[1]
lx_off_pred = lx_off_pred1 * 0.5 + lx_off_pred2 * 0.5
lx_y_pred = lx_y_pred1*0.5+lx_y_pred2* 0.5

## 0.82 public

In [46]:
import polars as pl
import pandas as pd
import numpy as np
import re
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings("ignore")



num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def count_by_values(df, colname, values):
    fts = df.select(pl.col('id').unique(maintain_order=True))
    for i, value in enumerate(values):
        tmp_df = df.group_by('id').agg(pl.col(colname).is_in([value]).sum().alias(f'{colname}_{i}_cnt'))
        fts  = fts.join(tmp_df, on='id', how='left') 
    return fts


def dev_feats(df):
    
    print("< Count by values features >")
    
    feats = count_by_values(df, 'activity', activities)
    feats = feats.join(count_by_values(df, 'text_change', text_changes), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'down_event', events), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'up_event', events), on='id', how='left') 

    print("< Input words stats features >")

    temp = df.filter((~pl.col('text_change').str.contains('=>')) & (pl.col('text_change') != 'NoChange'))
    temp = temp.group_by('id').agg(pl.col('text_change').str.concat('').str.extract_all(r'q+'))
    temp = temp.with_columns(input_word_count = pl.col('text_change').list.lengths(),
                             input_word_length_mean = pl.col('text_change').apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_max = pl.col('text_change').apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_std = pl.col('text_change').apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_median = pl.col('text_change').apply(lambda x: np.median([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_skew = pl.col('text_change').apply(lambda x: skew([len(i) for i in x] if len(x) > 0 else 0)))
    temp = temp.drop('text_change')
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Numerical columns features >")

    temp = df.group_by("id").agg(pl.sum('action_time').suffix('_sum'), pl.mean(num_cols).suffix('_mean'), pl.std(num_cols).suffix('_std'),
                                 pl.median(num_cols).suffix('_median'), pl.min(num_cols).suffix('_min'), pl.max(num_cols).suffix('_max'),
                                 pl.quantile(num_cols, 0.5).suffix('_quantile'))
    feats = feats.join(temp, on='id', how='left') 


    print("< Categorical columns features >")
    
    temp  = df.group_by("id").agg(pl.n_unique(['activity', 'down_event', 'up_event', 'text_change']))
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Idle time features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.group_by("id").agg(inter_key_largest_lantency = pl.max('time_diff'),
                                   inter_key_median_lantency = pl.median('time_diff'),
                                   mean_pause_time = pl.mean('time_diff'),
                                   std_pause_time = pl.std('time_diff'),
                                   total_pause_time = pl.sum('time_diff'),
                                   pauses_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 0.5) & (pl.col('time_diff') < 1)).count(),
                                   pauses_1_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1) & (pl.col('time_diff') < 1.5)).count(),
                                   pauses_1_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1.5) & (pl.col('time_diff') < 2)).count(),
                                   pauses_2_sec = pl.col('time_diff').filter((pl.col('time_diff') > 2) & (pl.col('time_diff') < 3)).count(),
                                   pauses_3_sec = pl.col('time_diff').filter(pl.col('time_diff') > 3).count(),)
    feats = feats.join(temp, on='id', how='left') 
    
    print("< P-bursts features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('time_diff')<2)
    temp = temp.with_columns(pl.when(pl.col("time_diff") & pl.col("time_diff").is_last()).then(pl.count()).over(pl.col("time_diff").rle_id()).alias('P-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('P-bursts').suffix('_mean'), pl.std('P-bursts').suffix('_std'), pl.count('P-bursts').suffix('_count'),
                                   pl.median('P-bursts').suffix('_median'), pl.max('P-bursts').suffix('_max'),
                                   pl.first('P-bursts').suffix('_first'), pl.last('P-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left') 


    print("< R-bursts features >")

    temp = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('activity').is_in(['Remove/Cut']))
    temp = temp.with_columns(pl.when(pl.col("activity") & pl.col("activity").is_last()).then(pl.count()).over(pl.col("activity").rle_id()).alias('R-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('R-bursts').suffix('_mean'), pl.std('R-bursts').suffix('_std'), 
                                   pl.median('R-bursts').suffix('_median'), pl.max('R-bursts').suffix('_max'),
                                   pl.first('R-bursts').suffix('_first'), pl.last('R-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left')
    
    return feats


def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

AGGREGATIONS = ['count', 'mean', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']

def reconstruct_essay(currTextInput):
    essayText = ""
    for Input in currTextInput.values:
        if Input[0] == 'Replace':
            replaceTxt = Input[2].split(' => ')
            essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
            continue
        if Input[0] == 'Paste':
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
            continue
        if Input[0] == 'Remove/Cut':
            essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
            continue
        if "M" in Input[0]:
            croppedTxt = Input[0][10:]
            splitTxt = croppedTxt.split(' To ')
            valueArr = [item.split(', ') for item in splitTxt]
            moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
            if moveData[0] != moveData[2]:
                if moveData[0] < moveData[2]:
                    essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                else:
                    essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
            continue
        essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
    return essayText


def get_essay_df(df):
    df       = df[df.activity != 'Nonproduction']
    temp     = df.groupby('id').apply(lambda x: reconstruct_essay(x[['activity', 'cursor_position', 'text_change']]))
    essay_df = pd.DataFrame({'id': df['id'].unique().tolist()})
    essay_df = essay_df.merge(temp.rename('essay'), on='id')
    return essay_df


def word_feats(df):
    essay_df = df
    df['word'] = df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    df = df.explode('word')
    df['word_len'] = df['word'].apply(lambda x: len(x))
    df = df[df['word_len'] != 0]

    word_agg_df = df[['id','word_len']].groupby(['id']).agg(AGGREGATIONS)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df


def sent_feats(df):
    df['sent'] = df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    df = df.explode('sent')
    df['sent'] = df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    df['sent_len'] = df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    df['sent_word_count'] = df['sent'].apply(lambda x: len(x.split(' ')))
    df = df[df.sent_len!=0].reset_index(drop=True)

    sent_agg_df = pd.concat([df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), 
                             df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1)
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df


def parag_feats(df):
    df['paragraph'] = df['essay'].apply(lambda x: x.split('\n'))
    df = df.explode('paragraph')
    # Number of characters in paragraphs
    df['paragraph_len'] = df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    df['paragraph_word_count'] = df['paragraph'].apply(lambda x: len(x.split(' ')))
    df = df[df.paragraph_len!=0].reset_index(drop=True)
    
    paragraph_agg_df = pd.concat([df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), 
                                  df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

def product_to_keys(logs, essays):
    essays['product_len'] = essays.essay.str.len()
    tmp_df = logs[logs.activity.isin(['Input', 'Remove/Cut'])].groupby(['id']).agg({'activity': 'count'}).reset_index().rename(columns={'activity': 'keys_pressed'})
    essays = essays.merge(tmp_df, on='id', how='left')
    essays['product_to_keys'] = essays['product_len'] / essays['keys_pressed']
    return essays[['id', 'product_to_keys']]

def get_keys_pressed_per_second(logs):
    temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
    temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
    temp_df = temp_df.merge(temp_df_2, on='id', how='left')
    temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
    return temp_df[['id', 'keys_per_second']]


data_path     = '/kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(data_path + 'train_logs.csv')
train_feats   = dev_feats(train_logs)
train_feats   = train_feats.collect().to_pandas()

print('< Essay Reconstruction >')
train_logs             = train_logs.collect().to_pandas()
train_essays           = get_essay_df(train_logs)
train_feats            = train_feats.merge(word_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(sent_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(parag_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(get_keys_pressed_per_second(train_logs), on='id', how='left')
train_feats            = train_feats.merge(product_to_keys(train_logs, train_essays), on='id', how='left')


print('< Mapping >')
train_scores   = pd.read_csv(data_path + 'train_scores.csv')
data           = train_feats.merge(train_scores, on='id', how='left')
x              = data.drop(['id', 'score'], axis=1)
y              = data['score'].values
print(f'Number of features: {len(x.columns)}')


print('< Testing Data >')
test_logs   = pl.scan_csv(data_path + 'test_logs.csv')
test_feats  = dev_feats(test_logs)
test_feats  = test_feats.collect().to_pandas()

test_logs             = test_logs.collect().to_pandas()
test_essays           = get_essay_df(test_logs)
test_feats            = test_feats.merge(word_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(get_keys_pressed_per_second(test_logs), on='id', how='left')
test_feats            = test_feats.merge(product_to_keys(test_logs, test_essays), on='id', how='left')

test_ids = test_feats['id'].values
testin_x = test_feats.drop(['id'], axis=1)



n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
data["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(skf.split(data, data["score"].astype(str))):
    data.loc[val_idx, "fold"] = idx


from sklearn.metrics import mean_squared_error
rmse = lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False)



from collections import defaultdict

drop_cols = ["id", "score", "fold"]
oof_df = pd.DataFrame()
models = defaultdict(list)
models_to_ensemble = ["catboost", "xgboost", "lgbm"]
model_params = {
    "catboost": {
        "iterations": 5000,
        "early_stopping_rounds": 50,
        "depth": 6,
        "loss_function": "RMSE",
        "random_seed": 42,
        "silent": True
    },
    "lgbm": {
        'n_estimators': 1024,
        'learning_rate': 0.005,
        'metric': 'rmse',
        'random_state': 42,
        'force_col_wise': True,
        'verbosity': 0
    },
    "xgboost": {
        "max_depth": 4,
        "learning_rate": 0.1,
        "objective": "reg:squarederror",
        "num_estimators": 1000,
        "num_boost_round": 1000,
        "eval_metric": "rmse",
        "seed": 42
    },
}
OOF_DF = data[['id','score']]
n_splits = 5
for i in list(range(5)):
    for idx, model_name in enumerate(models_to_ensemble):
        params = model_params[model_name]
        oof_folds = np.zeros(len(data))
        print(f"Started the {model_name} model...")
        for j,fold in enumerate(range(n_splits)):
            if model_name == "lgbm":
                model = LGBMRegressor(**params)
            elif model_name == "xgboost":
                model = xgb.XGBRegressor(**params)
            elif model_name == "catboost":
                model = CatBoostRegressor(**params)
            else:
                raise ValueError("Unknown base model name.")
            save_model_name = model_name+f"_seed{i}"
            oof_cols_name_ = model_name+f"_seed{i}"
            if j==0:
                OOF_DF[f'{oof_cols_name_}_preds'] = np.zeros(len(data))
            skf = StratifiedKFold(n_splits=n_splits, random_state=42+i, shuffle=True)
            data["fold"] = -1
            for idx, (train_idx, val_idx) in enumerate(skf.split(data, data["score"].astype(str))):
                data.loc[val_idx, "fold"] = idx

            x_train = data[data["fold"] != fold].reset_index(drop=True)
            x_valid = data[data["fold"] == fold].reset_index(drop=True)
            valid_index = data[data["fold"] == fold].index
            y_train = x_train["score"]
            y_valid = x_valid["score"]
            ids = x_valid["id"]

            x_train = x_train.drop(drop_cols, axis="columns")
            x_valid = x_valid.drop(drop_cols, axis="columns")
            model.fit(x_train, y_train)
            val_preds = model.predict(x_valid)
            oof_fold = pd.concat([ids, y_valid, pd.Series(val_preds)],axis=1).rename({0: f"{oof_cols_name_}_preds"}, axis="columns")
            oof_folds[valid_index] = oof_fold[f"{oof_cols_name_}_preds"]
            models[save_model_name].append(model)
            print(f"Fold: {fold} - Score: {rmse(oof_fold['score'], oof_fold[f'{oof_cols_name_}_preds']):.5f}")
        OOF_DF[f"{oof_cols_name_}_preds"] = oof_folds
        cv_score = rmse(OOF_DF["score"], OOF_DF[f"{oof_cols_name_}_preds"])
        print(f"{oof_cols_name_} cv_score: ", round(cv_score, 5))


import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

pred_cols = [i for i in OOF_DF.columns if i not in ['id','score']]
true_targets = OOF_DF["score"]

def objective_function(weights):
    ensemble_preds = (OOF_DF[pred_cols] * weights).sum(axis=1)
    score = rmse(OOF_DF["score"], ensemble_preds)
    return score

def find_weights(oof_df):
    len_models = len(pred_cols)
    initial_weights = np.ones(len_models) / len_models
    print(initial_weights)
    bounds = [(0, 1)] * len_models
    result = minimize(objective_function, initial_weights, bounds=bounds, method='SLSQP') # L-BFGS-B
    optimized_weights = result.x
    optimized_weights /= np.sum(optimized_weights)
    return optimized_weights

optimized_weights = find_weights(OOF_DF)
print("Optimized Weights:", optimized_weights)

OOF_DF["ensemble_optimized_preds"] = (OOF_DF[pred_cols] * optimized_weights).sum(axis=1)
cv_optimized = rmse(OOF_DF["score"], OOF_DF["ensemble_optimized_preds"])
print("cv_score with optimized weights: ", round(cv_optimized, 5))

preds = None
MODELS = [f"{i}_seed{j}" for j in range(5) for i in models_to_ensemble]
for weights, model_name in zip(optimized_weights,MODELS):
    models_list = models[model_name]
    current_preds = weights * np.mean([model.predict(testin_x) for model in models_list], axis=0)
    if preds is None:
        preds = current_preds
    else:
        preds += current_preds

y_pred581 = preds

< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >
< Essay Reconstruction >
< Mapping >
Number of features: 165
< Testing Data >
< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >
Started the catboost model...
Fold: 0 - Score: 0.61398
Fold: 1 - Score: 0.59990
Fold: 2 - Score: 0.60852
Fold: 3 - Score: 0.62396
Fold: 4 - Score: 0.65914
catboost_seed0 cv_score:  0.62144
Started the xgboost model...
Fold: 0 - Score: 0.60045
Fold: 1 - Score: 0.60438
Fold: 2 - Score: 0.60173
Fold: 3 - Score: 0.61313
Fold: 4 - Score: 0.66174
xgboost_seed0 cv_score:  0.61671
Started the lgbm model...
Fold: 0 - Score: 0.60417
Fold: 1 - Score: 0.59835
Fold: 2 - Score: 0.59072
Fold: 3 - Score: 0.61666
Fold: 4 - Score: 0.67026
lgbm_seed0 cv

## blending

In [47]:
optimize_cv = False
if optimize_cv:
    def objective_function(weights):
        ensemble_preds = (OOF_PREDS_ALL[['LGBM_pandas','MLP','LGMB_polars1','LGBM_polars2','LGBM_polars_public']] * weights).sum(axis=1)
        score = rmse(OOF_PREDS_ALL["score"], ensemble_preds)
        return score

    def find_weights(oof_df):
        len_models = len(['LGBM_pandas','MLP','LGMB_polars1','LGBM_polars2','LGBM_polars_public'])
        initial_weights = np.ones(len_models) / len_models
        print(initial_weights)
        bounds = [(0, 1)] * len_models
        result = minimize(objective_function, initial_weights, bounds=bounds, method='SLSQP') # L-BFGS-B
        optimized_weights = result.x
        optimized_weights /= np.sum(optimized_weights)
        return optimized_weights

    OOF_PREDS_ALL = pd.DataFrame()
    OOF_PREDS_ALL['score'] = train_scores['score']
    OOF_PREDS_ALL['LGBM_pandas'] = OOF_PREDS_lgbm_pandas
    OOF_PREDS_ALL['MLP'] = OOF_PREDS_mlp
    OOF_PREDS_ALL['LGMB_polars1'] = lx_off_pred1
    OOF_PREDS_ALL['LGBM_polars2'] = lx_off_pred2
    OOF_PREDS_ALL['LGBM_polars_public'] = OOF_DF["ensemble_optimized_preds"]

    optimized_weights = find_weights(OOF_PREDS_ALL)
    print("Optimized Weights:", optimized_weights)

    OOF_PREDS_ALL["ensemble_all"] = (OOF_PREDS_ALL[['LGBM_pandas','MLP','LGMB_polars1','LGBM_polars2','LGBM_polars_public']] * optimized_weights).sum(axis=1)
    cv_optimized = rmse(OOF_PREDS_ALL["score"], OOF_PREDS_ALL["ensemble_all"])
    print('cv_optimized',cv_optimized)
    test_preds_blend = TEST_PREDS_lgbm_pandas * optimized_weights[0] + TEST_PREDS_mlp * optimized_weights[1] +lx_y_pred1 * optimized_weights[2] + lx_y_pred2 * optimized_weights[3] + y_pred581*optimized_weights[4]
    test_preds_blend
    
else:
    
    weights = [0.35,0.35,0.03,0.27]  
    OOF_PREDS_blend = lx_off_pred*weights[0] + OOF_PREDS_lgbm_pandas*weights[1] + OOF_DF["ensemble_optimized_preds"]*weights[2] + OOF_PREDS_mlp*weights[3]
    print('OOF metric blend = {:.5f}'.format(metrics.mean_squared_error(train_scores[target_col], OOF_PREDS_blend, squared=False)))
    test_preds_blend = lx_y_pred * weights[0] + TEST_PREDS_lgbm_pandas * weights[1] + y_pred581*weights[2] + TEST_PREDS_mlp*weights[3]
    test_preds_blend

OOF metric blend = 0.58983


In [48]:
test_feats['score'] = test_preds_blend
test_feats[['id', 'score']].to_csv("submission.csv", index=False)

In [49]:
test_feats[['id', 'score']]

,id,score
0,0000aaaa,1.362051
1,2222bbbb,1.116869
2,4444cccc,1.114732
